In [ ]:
# import pandas as pd
# from sklearn.metrics import mean_absolute_error

# # Load CSV
# df = pd.read_csv("MAE.csv")

# # Define pairs for comparison
# pairs = [
#     ("CRI_model", "CRI_actual"),
#     ("CSR_model", "CSR_actual"),
#     ("ASH_model", "ASH_actual"),
#     ("VM_model", "VM_actual"),

#     ("CRI_excel_model", "CRI_actual"),
#     ("CSR_excel_model", "CSR_actual"),
#     ("ASH_excel_model", "ASH_actual"),
#     ("VM_excel_model", "VM_actual"),

#     ("CRI_mean", "CRI_actual"),
#     ("CSR_mean", "CSR_actual")
# ]

# # Compute MAE for each pair
# results = {}

# for pred, actual in pairs:
#     mae = mean_absolute_error(df[actual], df[pred])
#     results[f"{pred} vs {actual}"] = mae

# # Convert to DataFrame for nice formatting
# mae_df = pd.DataFrame(list(results.items()), columns=["Comparison", "MAE"])

# print(mae_df)


                      Comparison       MAE
0        CRI_model vs CRI_actual  1.047670
1        CSR_model vs CSR_actual  1.604113
2        ASH_model vs ASH_actual  0.614379
3          VM_model vs VM_actual  0.048883
4  CRI_excel_model vs CRI_actual  1.041538
5  CSR_excel_model vs CSR_actual  1.268077
6  ASH_excel_model vs ASH_actual  0.921154
7    VM_excel_model vs VM_actual  0.121538
8         CRI_mean vs CRI_actual  1.038077
9         CSR_mean vs CSR_actual  1.548769


In [1]:
!pip uninstall -y pycaret
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [6]:
!pip install git+https://github.com/pycaret/pycaret.git@master


  Cloning https://github.com/pycaret/pycaret.git (to revision master) to /tmp/pip-req-build-obuuz2i3
  Running command git clone --filter=blob:none --quiet https://github.com/pycaret/pycaret.git /tmp/pip-req-build-obuuz2i3
  Resolved https://github.com/pycaret/pycaret.git to commit 58ec3c282d58e94727f9d5b77b49f241e9103ab3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
pip install deap catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 45.9 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [catboost]1/2 [catboost]


In [7]:
import pandas as pd
import numpy as np
import random
from typing import Dict, List, Tuple
from copy import deepcopy
import pandas as pd
import numpy as np
import pickle
import re
from pathlib import Path
from pycaret.regression import load_model, predict_model


class CoalBlendPredictor:
    """
    Coal Blend Prediction System

    Predicts CRI, CSR, ASH, and VM values based on coal blend ratios.
    """

    def __init__(self, coal_db_path="Clean_exist_coals.csv", models_dir="."):
        """
        Initialize the Coal Blend Predictor

        Parameters:
        -----------
        coal_db_path : str
            Path to Clean_exist_coals.csv
        scaler_path : str
            Path to scaler_colwise.pkl
        models_dir : str
            Directory containing model files (CRI_model.pkl, etc.)
        """
        self.coal_db_path = coal_db_path
        self.models_dir = Path(models_dir)

        # Initialize coal database
        self.coal_db = None
        self.feature_names = None
        self.models = {}
        self.model_features = {}

        # Load all necessary files
        self._load_resources()

    def _load_resources(self):
        """Load coal database, scaler, and models"""
        try:
            # Load coal database
            self.coal_db = self._load_coal_database()
            self.coal_db = self._feature_engineering(self.coal_db)
            self.coal_db = self._prepare_coal_properties(self.coal_db)

            # Load scaler
            # with open(self.scaler_path, "rb") as f:
            #     scaler_data = pickle.load(f)
            # self.scaler = scaler_data['scaler']
            # self.feature_names = scaler_data['feature_names']

            # Load models
            targets = ["CRI", "CSR", "ASH", "VM"]
            for target in targets:
                model_path = self.models_dir / f"{target}_model"
                features_path = self.models_dir / f"{target}_features.pkl"

                self.models[target] = load_model(str(model_path))

                with open(features_path, "rb") as f:
                    self.model_features[target] = pickle.load(f)

        except Exception as e:
            raise RuntimeError(f"Failed to load resources: {str(e)}")

    @staticmethod
    def _normalize_name(s):
        """Normalize coal names for matching"""
        s = str(s).lower().strip()
        s = re.sub(r"[^a-z0-9]+", " ", s)
        s = re.sub(r"\s+", " ", s).strip()
        return s

    def _load_coal_database(self):
        """Load and prepare coal properties database"""
        df = pd.read_csv(self.coal_db_path)

        # Clean column names
        df.columns = df.columns.str.strip()
        df.columns = df.columns.str.replace(r"\s+", " ", regex=True)
        df.columns = df.columns.str.replace("\ufeff", "")

        # Fill missing values
        df = df.fillna(0)

        # Create normalized name column for matching
        df["__norm_name"] = df["Name_of_coal"].map(self._normalize_name)

        return df

    def _feature_engineering(self, df):
        """Apply feature engineering (BI, MBI, VR, CBI, Log_Max_Fluidity)"""

        # 1. Calculate BI
        df["BI"] = (df["Fe2O3"] + df["CaO"] + df["MgO"] + df["Na2O"] + df["K2O"]) / (
            df["SiO2"] + df["Al2O3"] + df["TiO2"]
        )

        # 2. Calculate MBI
        df["MBI"] = ((df["Ash"] * 100) / (100 - df["V.M."])) * df["BI"]

        # 3. VR7 to VR19
        ratios = {
            "V7": 3,
            "V8": 2.7,
            "V9": 2.5,
            "V10": 2.4,
            "V11": 2.5,
            "V12": 3,
            "V13": 3.7,
            "V14": 5,
            "V15": 7,
            "V16": 9.6,
            "V17": 12,
            "V18": 15,
            "V19": 18,
        }

        for col, divisor in ratios.items():
            if col in df.columns:
                df[f"VR{col[1:]}"] = df[col] / divisor

        # 4. Calculate CBI only for HCC, HFCC, SHCC
        VR_cols = [f"VR{i}" for i in range(7, 20) if f"VR{i}" in df.columns]
        sum_vr = df[VR_cols].sum(axis=1)

        df["CBI"] = np.where(
            (df["Coal Category"].isin(["HCC", "HFCC", "SHCC"])) & (sum_vr != 0),
            (df["Inertinite"] + df["Minerals"]) / sum_vr,
            0,
        )

        # 5. Log of Max. Fluidity ddpm
        if "Max. Fluidity ddpm" in df.columns:
            df["Log_Max_Fluidity"] = np.log(
                df["Max. Fluidity ddpm"].replace(0, np.nan)
            ).fillna(0)

        return df

    def _prepare_coal_properties(self, df):
        """Prepare coal properties after feature engineering"""

        # Columns to drop
        cols_to_drop = (
            ["Max. Fluidity ddpm"]
            + [f"V{i}" for i in range(7, 20)]
            + [f"VR{i}" for i in range(7, 20)]
            + [
                "Inertinite",
                "Minerals",
                "SiO2",
                "Al2O3",
                "Fe2O3",
                "CaO",
                "MgO",
                "Na2O",
                "K2O",
                "TiO2",
                "Mn3O4",
                "SO3",
                "P2O5",
                "BaO",
                "SrO",
                "ZnO",
            ]
            + ["C", "H", "O", "N", "S"]
            + [
                "HGI (ASTM)",
                "Rank",
                "Vitrinite %",
                "Liptinite",
                "Semi-Fusinite",
                "Initial Softening Temp.",
            ]
        )

        # Drop only existing columns
        df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

        return df

    def _calculate_weighted_features(self, coal_inputs):
        """
        Calculate weighted features based on coal inputs

        Parameters:
        -----------
        coal_inputs : dict
            Dictionary like {'Goonyella': 50, 'R.PCI': 50}

        Returns:
        --------
        dict : Category percentages and weighted features
        """

        # Initialize category percentages
        categories = {"HCC": 0, "SHCC": 0, "HFCC": 0, "PCI": 0, "WC": 0}

        # Property columns (exclude Name and Category)
        exclude_cols = ["Name_of_coal", "__norm_name", "Coal Category"]
        prop_cols = [c for c in self.coal_db.columns if c not in exclude_cols]

        # Initialize weighted sums
        weighted_features = {f"weighted_{col}": 0 for col in prop_cols}

        # Process each coal input
        for coal_name, ratio in coal_inputs.items():
            norm_name = self._normalize_name(coal_name)

            # Find coal in database
            coal_row = self.coal_db[self.coal_db["__norm_name"] == norm_name]

            if coal_row.empty:
                raise ValueError(f"Coal '{coal_name}' not found in database!")

            coal_row = coal_row.iloc[0]

            # Add to category
            category = coal_row["Coal Category"]
            if category in categories:
                categories[category] += ratio

            # Calculate weighted properties
            for col in prop_cols:
                weighted_features[f"weighted_{col}"] += coal_row[col] * ratio

        # Combine categories and weighted features
        result = {**categories, **weighted_features}

        return result

    # def _calculate_cri_csr(self, blend_features):
    #     """Calculate CRI and CSR using formulas"""

    #     vm = blend_features['weighted_V.M.']
    #     log_fluidity = blend_features['weighted_Log_Max_Fluidity']

    #     # Calculate CRI
    #     cri_direct = -16.48 + 8.16 * vm - 21.68 * log_fluidity

    #     # Calculate CSR (two formulas)
    #     csr_from_cri = 94.19 - 1.15 * cri_direct
    #     csr_direct = 95.76 - 2.50 * vm + 11.00 * log_fluidity

    #     blend_features['CRI_direct'] = cri_direct
    #     blend_features['CSR_from_CRI'] = csr_from_cri
    #     blend_features['CSR_direct'] = csr_direct

    #     return blend_features

    # def _normalize_features(self, blend_features):
    #     """Normalize features using loaded scaler"""

    #     # Extract feature values in correct order
    #     feature_values = [blend_features[fname] for fname in self.feature_names]
    #     feature_array = np.array(feature_values).reshape(1, -1)

    #     # Normalize (column-wise)
    #     normalized = self.scaler.transform(feature_array)

    #     # Create normalized dict
    #     normalized_dict = dict(zip(self.feature_names, normalized[0]))

    #     return normalized_dict

    def _predict_targets(self, normalized_features):
        """Predict CRI, CSR, ASH, VM using loaded models"""

        predictions = {}

        for target in ["CRI", "CSR", "ASH", "VM"]:
            # Prepare input DataFrame
            input_df = pd.DataFrame([normalized_features])
            input_df = input_df[self.model_features[target]]

            # Predict
            pred = predict_model(self.models[target], data=input_df)
            predictions[target] = pred["prediction_label"].iloc[0]

        return predictions

    def predict(self, coal_inputs, verbose=False):
        """
        Main prediction method

        Parameters:
        -----------
        coal_inputs : dict
            Dictionary of coal names and their ratios
            Example: {'Goonyella': 50, 'R.PCI': 50}
        verbose : bool
            If True, print detailed information

        Returns:
        --------
        dict : Contains blend_composition, category_distribution, and predictions

        Example:
        --------
        >>> predictor = CoalBlendPredictor()
        >>> result = predictor.predict({'Goonyella': 50, 'R.PCI': 50})
        >>> print(result['predictions']['CRI'])
        27.5
        """

        if verbose:
            print("🔮 Starting prediction...")

        # Validate total ratio
        total_ratio = sum(coal_inputs.values())
        if abs(total_ratio - 100) > 0.01:
            raise ValueError(f"Total ratio must be 100%, got {total_ratio}%")

        try:
            # Calculate weighted features
            blend_features = self._calculate_weighted_features(coal_inputs)

            # Calculate CRI/CSR
            # blend_features = self._calculate_cri_csr(blend_features)

            # # Normalize features
            # normalized_features = self._normalize_features(blend_features)

            # Predict targets
            predictions = self._predict_targets(blend_features)

            # Prepare result
            result = {
                "blend_composition": coal_inputs,
                "category_distribution": {
                    k: blend_features[k] for k in ["HCC", "SHCC", "HFCC", "PCI", "WC"]
                },
                "predictions": predictions,
                "status": "success",
            }

            if verbose:
                self._print_results(result)
            print(result)
            return result

        except Exception as e:
            return {
                "status": "error",
                "error_message": str(e),
                "blend_composition": coal_inputs,
            }

    def _print_results(self, result):
        """Print formatted results"""
        print("\n" + "=" * 60)
        print("BLEND COMPOSITION")
        print("=" * 60)
        for coal, ratio in result["blend_composition"].items():
            print(f"  {coal}: {ratio}%")

        print("\n" + "=" * 60)
        print("COAL CATEGORY DISTRIBUTION")
        print("=" * 60)
        for cat, value in result["category_distribution"].items():
            print(f"  {cat}: {value}%")

        print("\n" + "=" * 60)
        print("PREDICTIONS")
        print("=" * 60)
        for target, value in result["predictions"].items():
            print(f"  {target}: {value:.2f}")
        print("=" * 60 + "\n")

    def batch_predict(self, coal_inputs_list, verbose=False):
        """
        Predict for multiple blends

        Parameters:
        -----------
        coal_inputs_list : list of dict
            List of coal input dictionaries
        verbose : bool
            If True, print progress

        Returns:
        --------
        list : List of prediction results
        """
        results = []

        for i, coal_inputs in enumerate(coal_inputs_list):
            if verbose:
                print(f"\n[{i+1}/{len(coal_inputs_list)}] Processing blend...")

            result = self.predict(coal_inputs, verbose=verbose)
            results.append(result)

        return results

    def get_available_coals(self):
        """
        Get list of available coal names in the database

        Returns:
        --------
        list : List of coal names
        """
        return self.coal_db["Name_of_coal"].tolist()

    def get_coal_info(self, coal_name):
        """
        Get information about a specific coal

        Parameters:
        -----------
        coal_name : str
            Name of the coal

        Returns:
        --------
        dict : Coal properties and category
        """
        norm_name = self._normalize_name(coal_name)
        coal_row = self.coal_db[self.coal_db["__norm_name"] == norm_name]

        if coal_row.empty:
            return None

        coal_row = coal_row.iloc[0]
        return {
            "name": coal_row["Name_of_coal"],
            "category": coal_row["Coal Category"],
            "properties": coal_row.drop(
                ["Name_of_coal", "__norm_name", "Coal Category"]
            ).to_dict(),
        }

    def Blend_cost(self, blend_dict):
        # Load coal cost data
        coal_costs = pd.read_csv("/content/coal_costs.csv")
        coal_costs.set_index("Name_of_coal", inplace=True)
        total_cost = 0.0
        for coal_name, percentage in blend_dict.items():
            if coal_name not in coal_costs.index:
                raise ValueError(f"Coal '{coal_name}' not found in coal_costs.csv")
            cost_per_ton = coal_costs.loc[coal_name, "Cost"]
            ton_fraction = percentage / 100  # convert % to fraction
            total_cost += cost_per_ton * ton_fraction
        return {"Total blend Cost": float(total_cost)}

    """
            Below is the way to make input for cost function i.e cost_input
            predictions = model_result.get('predictions', {})
            cost_dict = predictor.Blend_cost(blend)
            cost_fn_input = {'Total blend Cost': cost_dict.get('Total blend Cost', 0.0), 'predictions': predictions}
            cost_result = predictor.cost_function(cost_fn_input)
    """

    def cost_function(self, cost_input):
        # combine difference from desired values + blend cost in a simple way
        desired = {
            "CRI": {"type": "range", "min": 22, "max": 28, "weight": 1.0},
            "CSR": {"type": "min", "min": 63, "weight": 1.5},
            "ASH": {"type": "max", "max": 13.5, "weight": 1.2},
        }
        preds = cost_input["predictions"]
        penalty = 0.0
        # CRI: penalty outside range
        d = desired["CRI"]
        if preds["CRI"] < d["min"]:
            penalty += (d["min"] - preds["CRI"]) * d["weight"] * 100
        elif preds["CRI"] > d["max"]:
            penalty += (preds["CRI"] - d["max"]) * d["weight"] * 100
        # CSR: penalty if below min
        d = desired["CSR"]
        if preds["CSR"] < d["min"]:
            penalty += (d["min"] - preds["CSR"]) * d["weight"] * 100
        # ASH: penalty if above max
        d = desired["ASH"]
        if preds["ASH"] > d["max"]:
            penalty += (preds["ASH"] - d["max"]) * d["weight"] * 100
        # plus blend cost influence
        total_cost = cost_input.get("Total blend Cost", 0)
        # combine
        score = total_cost + penalty
        return {"Cost function evaluation": float(score)}

    def predict_from_csv(self, input_csv_path, output_csv_path=None, verbose=True):
        """
        Predict from CSV file where columns are coal names and values are ratios

        Parameters:
        -----------
        input_csv_path : str
            Path to input CSV file with coal names as columns and ratios as values
        output_csv_path : str, optional
            Path to save output CSV with predictions appended
            If None, will add '_predictions' to input filename
        verbose : bool
            If True, print progress

        Returns:
        --------
        pd.DataFrame : DataFrame with original data + predictions

        Example CSV format:
        Goonyella,R.PCI,Peak Downs,CRI,CSR,ASH,VM
        50,30,20,,,,
        40,40,20,,,,
        """

        if verbose:
            print(f"\n{'='*60}")
            print("COAL BLEND CSV PREDICTION")
            print(f"{'='*60}")
            print(f"📂 Loading: {input_csv_path}")

        # Load CSV
        df = pd.read_csv(input_csv_path)

        # Clean column names
        df.columns = df.columns.str.strip()

        # Identify prediction columns (CRI, CSR, ASH, VM) if they exist
        prediction_cols = ["CRI", "CSR", "ASH", "VM"]
        existing_pred_cols = [col for col in prediction_cols if col in df.columns]

        # Get coal columns (all columns except prediction columns)
        coal_columns = [col for col in df.columns if col not in prediction_cols]

        if verbose:
            print(f"🔍 Found {len(coal_columns)} coal columns")
            print(f"📊 Processing {len(df)} blend(s)...")

        # Initialize prediction columns if they don't exist
        for col in prediction_cols:
            if col not in df.columns:
                df[col] = np.nan

        # Process each row
        successful = 0
        failed = 0
        errors = []

        for idx, row in df.iterrows():
            # Create coal input dictionary (only non-zero/non-empty values)
            coal_inputs = {}
            for coal_name in coal_columns:
                value = row[coal_name]
                # Handle empty strings, NaN, and zeros
                if pd.notna(value) and value != "" and float(value) != 0:
                    coal_inputs[coal_name] = float(value)

            # Skip if no valid coal inputs
            if not coal_inputs:
                if verbose:
                    print(f"  ⚠️  Row {idx+1}: No valid coal ratios found, skipping...")
                failed += 1
                errors.append(f"Row {idx+1}: No valid coal ratios")
                continue

            # Predict
            result = self.predict(coal_inputs, verbose=False)

            if result["status"] == "success":
                # Update predictions in DataFrame
                for target in prediction_cols:
                    df.at[idx, target] = result["predictions"][target]
                successful += 1

                if verbose:
                    blend_str = ", ".join([f"{k}:{v}%" for k, v in coal_inputs.items()])
                    print(f"  ✅ Row {idx+1}: {blend_str}")
                    print(
                        f"      → CRI={result['predictions']['CRI']:.2f}, "
                        f"CSR={result['predictions']['CSR']:.2f}, "
                        f"ASH={result['predictions']['ASH']:.2f}, "
                        f"VM={result['predictions']['VM']:.2f}"
                    )
            else:
                failed += 1
                errors.append(f"Row {idx+1}: {result['error_message']}")
                if verbose:
                    print(f"  ❌ Row {idx+1}: {result['error_message']}")

        # Save output
        if output_csv_path is None:
            # Generate output filename
            input_path = Path(input_csv_path)
            output_csv_path = (
                input_path.parent / f"{input_path.stem}_predictions{input_path.suffix}"
            )

        df.to_csv(output_csv_path, index=False)

        if verbose:
            print(f"\n{'='*60}")
            print("SUMMARY")
            print(f"{'='*60}")
            print(f"✅ Successful predictions: {successful}/{len(df)}")
            if failed > 0:
                print(f"❌ Failed predictions: {failed}/{len(df)}")
                print("\nErrors:")
                for error in errors:
                    print(f"  • {error}")
            print(f"\n💾 Output saved to: {output_csv_path}")
            print(f"{'='*60}\n")

        return df


class CoalBlendGA:
    """
    Genetic Algorithm for optimizing coal blend ratios with integer-based ratios.
    """

    def __init__(
        self,
        best_blend_csv: str,
        clean_exist_coals_csv: str,
        predictor,
        population_size: int = 150,
        generations: int = 75,
        crossover_rate: float = 0.8,
        mutation_rate: float = 0.3,
        elite_size: int = 5,
        use_integer_ratios: bool = True,
        diversity_threshold: float = 0.15,
    ):
        """
        Initialize the Genetic Algorithm.

        Args:
            best_blend_csv: Path to Best_blend_ratio.csv
            clean_exist_coals_csv: Path to Clean_exist_coals.csv
            predictor: Model predictor object with predict(), Blend_cost(), and cost_function() methods
            population_size: Number of individuals in population
            generations: Number of generations to evolve
            crossover_rate: Probability of crossover
            mutation_rate: Probability of mutation
            elite_size: Number of top individuals to preserve
            use_integer_ratios: If True, uses integer percentages (recommended)
            diversity_threshold: Minimum difference required between blends (0-1)
        """
        self.predictor = predictor
        self.population_size = population_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.use_integer_ratios = use_integer_ratios
        self.diversity_threshold = diversity_threshold

        # Load data
        self.initial_blends = pd.read_csv(best_blend_csv)
        self.coal_categories = pd.read_csv(clean_exist_coals_csv)

        # Extract coal names
        self.coal_names = list(self.initial_blends.columns)

        # Create category mapping
        self.category_map = dict(
            zip(
                self.coal_categories["Name_of_coal"],
                self.coal_categories["Coal Category"],
            )
        )

        # Define category constraints
        self.category_constraints = {
            "SHCC": {"highest_count": 3, "min_ratio": 20, "max_ratio": 52},
            "HCC": {"highest_count": 2, "min_ratio": 30, "max_ratio": 59},
            "HFCC": {"highest_count": 2, "min_ratio": 5, "max_ratio": 20},
            "WC": {"highest_count": 2, "min_ratio": 0, "max_ratio": 15},
            "PCI": {"highest_count": 1, "min_ratio": 10, "max_ratio": 15},
        }

        # Desired values for optimization
        self.desired_values = {
            "CRI": {"type": "range", "min": 22, "max": 28, "weight": 1.0},
            "CSR": {"type": "min", "min": 63, "weight": 1.5},
            "ASH": {"type": "max", "max": 13.5, "weight": 1.2},
        }

        self.population = []
        self.fitness_scores = []
        self.best_solution = None
        self.best_fitness = float("inf")
        self.history = []
        self.seen_blends = set()  # Track unique blends to avoid duplicates
        self.population_signatures = {}  # Map signature to fitness for quick lookup

    def calculate_diversity(self, blend1: np.ndarray, blend2: np.ndarray) -> float:
        """
        Calculate diversity between two blends (0 = identical, 1 = completely different).
        Uses normalized Manhattan distance.
        """
        blend1_norm = blend1 / 100.0  # Normalize to 0-1
        blend2_norm = blend2 / 100.0

        # Manhattan distance normalized by maximum possible distance
        distance = np.sum(np.abs(blend1_norm - blend2_norm)) / 2.0
        return distance

    def is_diverse_enough(
        self, blend: np.ndarray, population: List[np.ndarray]
    ) -> bool:
        """Check if blend is sufficiently different from existing population."""
        if len(population) == 0:
            return True

        for existing_blend in population:
            diversity = self.calculate_diversity(blend, existing_blend)
            if diversity < self.diversity_threshold:
                return False

        return True

    def normalize_to_integer(self, blend_array: np.ndarray) -> np.ndarray:
        """
        Normalize blend to sum to 100 using integer values.
        This eliminates decimal precision issues.
        """
        # Round to integers
        blend_int = np.round(blend_array).astype(int)

        # Ensure sum is exactly 100
        current_sum = np.sum(blend_int)
        diff = 100 - current_sum

        if diff != 0:
            # Find non-zero indices to adjust
            non_zero_indices = np.where(blend_int > 0)[0]

            if len(non_zero_indices) > 0:
                # Distribute the difference
                if diff > 0:
                    # Need to add
                    for _ in range(abs(diff)):
                        idx = random.choice(non_zero_indices)
                        blend_int[idx] += 1
                else:
                    # Need to subtract
                    for _ in range(abs(diff)):
                        # Only subtract from coals with ratio > 1
                        valid_indices = [
                            i for i in non_zero_indices if blend_int[i] > 1
                        ]
                        if valid_indices:
                            idx = random.choice(valid_indices)
                            blend_int[idx] -= 1
                        else:
                            break

        return blend_int.astype(float)

    def blend_to_signature(self, blend_array: np.ndarray) -> str:
        """Create a unique signature for a blend to detect duplicates."""
        blend_int = np.round(blend_array).astype(int)
        return ",".join(map(str, blend_int))

    def blend_array_to_dict(self, blend_array: np.ndarray) -> Dict[str, float]:
        """Convert blend array to dictionary format."""
        if self.use_integer_ratios:
            blend_array = np.round(blend_array).astype(int)
        return {
            coal: float(ratio)
            for coal, ratio in zip(self.coal_names, blend_array)
            if ratio > 0
        }

    def blend_dict_to_array(self, blend_dict: Dict[str, float]) -> np.ndarray:
        """Convert blend dictionary to array format."""
        array = np.zeros(len(self.coal_names))
        for i, coal in enumerate(self.coal_names):
            array[i] = blend_dict.get(coal, 0)
        if self.use_integer_ratios:
            array = self.normalize_to_integer(array)
        return array

    def check_constraints(
        self, blend_array: np.ndarray, check_soft: bool = True
    ) -> bool:
        """
        Check if blend satisfies all constraints.

        Args:
            blend_array: Array of coal blend ratios
            check_soft: If True, also check soft constraints
        """
        if self.use_integer_ratios:
            blend_array = np.round(blend_array).astype(int)

        # Check sum equals 100
        if abs(np.sum(blend_array) - 100) > 0.5:
            return False

        # Calculate weighted properties
        properties = {}
        for prop in ["PCI", "WC", "HCC", "SHCC", "HFCC"]:
            if hasattr(self, f"{prop.lower()}_values"):
                values = getattr(self, f"{prop.lower()}_values")
                properties[prop] = np.sum(blend_array * values) / 100

        # Hard constraints
        if "PCI" in properties and properties["PCI"] < 15:
            return False

        if "WC" in properties and properties["WC"] > 8:
            return False

        if "HCC" in properties and properties["HCC"] < 30:
            return False

        if "SHCC" in properties and properties["SHCC"] < 20:
            return False

        # Soft constraints (optional check)
        if check_soft:
            if "HCC" in properties and "SHCC" in properties:
                if properties["HCC"] + properties["SHCC"] < 60:
                    return False

            if "PCI" in properties and "WC" in properties:
                if properties["PCI"] + properties["WC"] > 15:
                    return False

            if "SHCC" in properties and "HFCC" in properties:
                if properties["SHCC"] < properties["HFCC"]:
                    return False

        # Check category constraints
        category_ratios = {}
        category_counts = {}

        for i, coal in enumerate(self.coal_names):
            if blend_array[i] > 0:
                category = self.category_map.get(coal, "Unknown")
                if category == "Unknown":
                    continue

                category_ratios[category] = (
                    category_ratios.get(category, 0) + blend_array[i]
                )
                category_counts[category] = category_counts.get(category, 0) + 1

        # Validate each category
        for category, constraints in self.category_constraints.items():
            ratio = category_ratios.get(category, 0)
            count = category_counts.get(category, 0)

            # If category is used, check constraints
            if ratio > 0:
                if count > constraints["highest_count"]:
                    return False
                if ratio < constraints["min_ratio"] or ratio > constraints["max_ratio"]:
                    return False

        return True

    def generate_random_blend(
        self, max_attempts: int = 1000, prioritize_soft: bool = True
    ) -> np.ndarray:
        """
        Generate a random blend with integer ratios satisfying constraints.

        Args:
            max_attempts: Maximum number of attempts to generate valid blend
            prioritize_soft: If True, try to satisfy soft constraints
        """
        for attempt in range(max_attempts):
            blend_array = np.zeros(len(self.coal_names))

            # Target ratios for each category
            category_targets = {}

            for category, constraints in self.category_constraints.items():
                category_coals = [
                    coal
                    for coal in self.coal_names
                    if self.category_map.get(coal) == category
                ]

                if not category_coals:
                    continue

                # Randomly decide how many coals to use
                max_coals = min(constraints["highest_count"], len(category_coals))
                num_coals = random.randint(1, max_coals) if max_coals > 1 else 1
                selected_coals = random.sample(category_coals, num_coals)

                # Random ratio within constraints (integer)
                min_ratio = constraints["min_ratio"]
                max_ratio = constraints["max_ratio"]
                category_ratio = (
                    random.randint(min_ratio, max_ratio)
                    if max_ratio > min_ratio
                    else min_ratio
                )
                category_targets[category] = (selected_coals, category_ratio)

            # Distribute ratios
            total_target = sum(target[1] for target in category_targets.values())

            if total_target > 0:
                # Scale to approximately 100
                scale_factor = 100 / total_target

                for category, (coals, ratio) in category_targets.items():
                    scaled_ratio = int(round(ratio * scale_factor))

                    # Distribute among coals in category
                    num_coals = len(coals)
                    base_ratio = scaled_ratio // num_coals
                    remainder = scaled_ratio % num_coals

                    for i, coal in enumerate(coals):
                        idx = self.coal_names.index(coal)
                        blend_array[idx] = base_ratio + (1 if i < remainder else 0)

            # Normalize to exactly 100
            blend_array = self.normalize_to_integer(blend_array)

            # Check hard constraints (and optionally soft)
            if self.check_constraints(blend_array, check_soft=False):
                # If we want soft constraints, check those too
                if prioritize_soft:
                    if self.check_constraints(blend_array, check_soft=True):
                        return blend_array
                    # Continue trying if soft constraints not met
                else:
                    return blend_array

        # If max_attempts reached and prioritize_soft was True,
        # try once more with only hard constraints
        if prioritize_soft:
            for attempt in range(max_attempts):
                blend_array = self._generate_blend_once()
                if self.check_constraints(blend_array, check_soft=False):
                    return blend_array

        # Return best effort blend
        return self.normalize_to_integer(np.zeros(len(self.coal_names)))


    # def _generate_blend_once(self) -> np.ndarray:
    #     """Helper method to generate a single random blend."""
    #     blend_array = np.zeros(len(self.coal_names))
    #     category_targets = {}

    #     for category, constraints in self.category_constraints.items():
    #         category_coals = [
    #             coal for coal in self.coal_names if self.category_map.get(coal) == category
    #         ]

    #         if not category_coals:
    #             continue

    #         max_coals = min(constraints["highest_count"], len(category_coals))
    #         num_coals = random.randint(1, max_coals) if max_coals > 1 else 1
    #         selected_coals = random.sample(category_coals, num_coals)

    #         min_ratio = constraints["min_ratio"]
    #         max_ratio = constraints["max_ratio"]
    #         category_ratio = (
    #             random.randint(min_ratio, max_ratio) if max_ratio > min_ratio else min_ratio
    #         )
    #         category_targets[category] = (selected_coals, category_ratio)

    #     total_target = sum(target[1] for target in category_targets.values())

    #     if total_target > 0:
    #         scale_factor = 100 / total_target

    #         for category, (coals, ratio) in category_targets.items():
    #             scaled_ratio = int(round(ratio * scale_factor))
    #             num_coals = len(coals)
    #             base_ratio = scaled_ratio // num_coals
    #             remainder = scaled_ratio % num_coals

    #             for i, coal in enumerate(coals):
    #                 idx = self.coal_names.index(coal)
    #                 blend_array[idx] = base_ratio + (1 if i < remainder else 0)

    #     return self.normalize_to_integer(blend_array)
    def repair_blend(self, blend_array: np.ndarray, max_iterations: int = 100) -> np.ndarray:
        """
        Repair a blend to satisfy all constraints.

        Args:
            blend_array: Array of coal blend ratios to repair
            max_iterations: Maximum number of repair attempts

        Returns:
            Repaired blend array
        """
        if self.use_integer_ratios:
            blend_array = np.round(blend_array).astype(int)

        for iteration in range(max_iterations):
            # First normalize to sum = 100
            blend_array = self.normalize_to_integer(blend_array)

            # Check if already valid
            if self.check_constraints(blend_array):
                return blend_array

            # Calculate current properties
            properties = {}
            for prop in ['PCI', 'WC', 'HCC', 'SHCC', 'HFCC']:
                prop_attr = f'{prop.lower()}_values'
                if hasattr(self, prop_attr):
                    values = getattr(self, prop_attr)
                    properties[prop] = np.sum(blend_array * values) / 100

            # Repair hard constraints by adjusting blend
            adjusted = False

            # Fix PCI > 10: reduce high-PCI coals
            if 'PCI' in properties and properties['PCI'] > 10:
                pci_values = self.pci_values
                high_pci_indices = np.where((blend_array > 0) & (pci_values > properties['PCI']))[0]
                if len(high_pci_indices) > 0:
                    idx = np.random.choice(high_pci_indices)
                    blend_array[idx] = max(0, blend_array[idx] - 1)
                    adjusted = True

            # Fix WC > 8: reduce high-WC coals
            if 'WC' in properties and properties['WC'] > 8:
                wc_values = self.wc_values
                high_wc_indices = np.where((blend_array > 0) & (wc_values > properties['WC']))[0]
                if len(high_wc_indices) > 0:
                    idx = np.random.choice(high_wc_indices)
                    blend_array[idx] = max(0, blend_array[idx] - 1)
                    adjusted = True

            # Fix HCC < 30: increase high-HCC coals
            if 'HCC' in properties and properties['HCC'] < 30:
                hcc_values = self.hcc_values
                high_hcc_indices = np.where(hcc_values > properties['HCC'])[0]
                if len(high_hcc_indices) > 0:
                    idx = np.random.choice(high_hcc_indices)
                    blend_array[idx] = blend_array[idx] + 1
                    adjusted = True

            # Fix SHCC < 20: increase high-SHCC coals
            if 'SHCC' in properties and properties['SHCC'] < 20:
                shcc_values = self.shcc_values
                high_shcc_indices = np.where(shcc_values > properties['SHCC'])[0]
                if len(high_shcc_indices) > 0:
                    idx = np.random.choice(high_shcc_indices)
                    blend_array[idx] = blend_array[idx] + 1
                    adjusted = True

            # Fix category constraints
            category_ratios = {}
            category_counts = {}
            category_coals = {}

            for i, coal in enumerate(self.coal_names):
                if blend_array[i] > 0:
                    category = self.category_map.get(coal, 'Unknown')
                    if category != 'Unknown':
                        category_ratios[category] = category_ratios.get(category, 0) + blend_array[i]
                        category_counts[category] = category_counts.get(category, 0) + 1
                        if category not in category_coals:
                            category_coals[category] = []
                        category_coals[category].append(i)

            for category, constraints in self.category_constraints.items():
                ratio = category_ratios.get(category, 0)
                count = category_counts.get(category, 0)

                if ratio > 0:
                    # Fix count violations
                    if count > constraints['highest_count']:
                        coal_indices = category_coals[category]
                        # Remove coal with smallest ratio
                        min_idx = min(coal_indices, key=lambda i: blend_array[i])
                        blend_array[min_idx] = 0
                        adjusted = True

                    # Fix ratio violations
                    if ratio < constraints['min_ratio']:
                        # Increase this category
                        if category in category_coals:
                            idx = np.random.choice(category_coals[category])
                            blend_array[idx] = blend_array[idx] + 1
                            adjusted = True

                    if ratio > constraints['max_ratio']:
                        # Decrease this category
                        if category in category_coals:
                            idx = np.random.choice(category_coals[category])
                            blend_array[idx] = max(0, blend_array[idx] - 1)
                            adjusted = True

            if not adjusted:
                # If no adjustment was made, we're stuck
                break

        # Final normalization
        blend_array = self.normalize_to_integer(blend_array)
        return blend_array

    def initialize_population(self) -> List[np.ndarray]:
        """Initialize population with diverse feasible blends."""
        population = []

        # Add initial blends from CSV (converted to integers) - but check diversity
        for idx, row in self.initial_blends.iterrows():
            blend_array = row.values.astype(float)
            if self.use_integer_ratios:
                blend_array = self.normalize_to_integer(blend_array)

            if self.check_constraints(blend_array):
                sig = self.blend_to_signature(blend_array)
                if sig not in self.seen_blends:
                    # Only add if diverse enough
                    if self.is_diverse_enough(blend_array, population):
                        population.append(blend_array)
                        self.seen_blends.add(sig)

        print(f"Added {len(population)} diverse blends from initial CSV")

        # Generate diverse random feasible blends
        max_attempts = self.population_size * 30
        attempts = 0

        while len(population) < self.population_size and attempts < max_attempts:
            blend_array = self.generate_random_blend()

            if self.check_constraints(blend_array):
                sig = self.blend_to_signature(blend_array)
                if sig not in self.seen_blends and self.is_diverse_enough(
                    blend_array, population
                ):
                    population.append(blend_array)
                    self.seen_blends.add(sig)

            attempts += 1

        print(f"Generated {len(population) - len(population)} random diverse blends")

        # If we don't have enough, generate with more aggressive mutation (relax diversity slightly)
        attempts = 0
        max_fill_attempts = self.population_size * 20

        while len(population) < self.population_size and attempts < max_fill_attempts:
            if len(population) > 0:
                base_blend = population[random.randint(0, len(population) - 1)].copy()
                mutated = self.aggressive_mutate(base_blend)

                if self.check_constraints(mutated):
                    sig = self.blend_to_signature(mutated)
                    if sig not in self.seen_blends:
                        population.append(mutated)
                        self.seen_blends.add(sig)
            else:
                # Generate random if no valid blends yet
                blend = self.generate_random_blend()
                if self.check_constraints(blend):
                    population.append(blend)

            attempts += 1

        return population

    def evaluate_fitness(self, blend_array: np.ndarray) -> float:
        """Evaluate fitness of a blend using cost function."""
        try:
            blend_dict = self.blend_array_to_dict(blend_array)

            if not blend_dict:
                return float("inf")

            # Get predictions
            model_result = self.predictor.predict(blend_dict, verbose=False)

            if model_result["status"] != "success":
                return float("inf")

            # Get blend cost
            blend_cost_result = self.predictor.Blend_cost(blend_dict)

            # Calculate cost function
            cost_input = {
                "Total blend Cost": blend_cost_result["Total blend Cost"],
                "predictions": model_result["predictions"],
            }

            cost_result = self.predictor.cost_function(cost_input)

            return cost_result["Cost function evaluation"]

        except Exception as e:
            print(f"Error evaluating fitness: {e}")
            return float("inf")

    def tournament_selection(self, tournament_size: int = 5) -> np.ndarray:
        """Select parent using tournament selection."""
        tournament_indices = random.sample(range(len(self.population)), tournament_size)
        tournament_fitness = [self.fitness_scores[i] for i in tournament_indices]
        winner_idx = tournament_indices[np.argmin(tournament_fitness)]
        return self.population[winner_idx].copy()

    def crossover(
        self, parent1: np.ndarray, parent2: np.ndarray
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Perform integer-based crossover between two parents.
        """
        if random.random() > self.crossover_rate:
            return parent1.copy(), parent2.copy()

        # Convert to integers
        p1_int = np.round(parent1).astype(int)
        p2_int = np.round(parent2).astype(int)

        # Choose crossover method
        method = random.choice(
            ["uniform", "single_point", "two_point", "category_based"]
        )

        if method == "uniform":
            # Uniform crossover
            mask = np.random.rand(len(p1_int)) < 0.5
            offspring1 = np.where(mask, p1_int, p2_int).astype(float)
            offspring2 = np.where(mask, p2_int, p1_int).astype(float)

        elif method == "single_point":
            # Single point crossover
            min_point = 1
            max_point = len(p1_int) - 1
            point = (
                random.randint(min_point, max_point)
                if max_point > min_point
                else min_point
            )
            offspring1 = np.concatenate([p1_int[:point], p2_int[point:]]).astype(float)
            offspring2 = np.concatenate([p2_int[:point], p1_int[point:]]).astype(float)

        elif method == "two_point":
            # Two point crossover
            if len(p1_int) > 2:
                points = sorted(random.sample(range(1, len(p1_int)), 2))
                offspring1 = np.concatenate(
                    [
                        p1_int[: points[0]],
                        p2_int[points[0] : points[1]],
                        p1_int[points[1] :],
                    ]
                ).astype(float)
                offspring2 = np.concatenate(
                    [
                        p2_int[: points[0]],
                        p1_int[points[0] : points[1]],
                        p2_int[points[1] :],
                    ]
                ).astype(float)
            else:
                # Fallback to uniform if not enough genes
                mask = np.random.rand(len(p1_int)) < 0.5
                offspring1 = np.where(mask, p1_int, p2_int).astype(float)
                offspring2 = np.where(mask, p2_int, p1_int).astype(float)

        else:  # category_based
            # Mix entire categories from parents
            offspring1 = p1_int.copy().astype(float)
            offspring2 = p2_int.copy().astype(float)

            for category in self.category_constraints.keys():
                if random.random() < 0.5:
                    # Swap this category between offspring
                    for i, coal in enumerate(self.coal_names):
                        if self.category_map.get(coal) == category:
                            offspring1[i], offspring2[i] = p2_int[i], p1_int[i]

        # Normalize to 100
        offspring1 = self.normalize_to_integer(offspring1)
        offspring2 = self.normalize_to_integer(offspring2)

        return offspring1, offspring2

    def mutate(self, blend_array: np.ndarray) -> np.ndarray:
        """Mutate a blend array with integer operations."""
        if random.random() > self.mutation_rate:
            return blend_array

        mutated = np.round(blend_array).astype(int).astype(float)

        # Choose mutation strategy
        strategy = random.choice(["swap", "adjust", "redistribute", "add_remove"])

        if strategy == "swap":
            # Swap ratios between two coals
            non_zero_indices = np.where(mutated > 0)[0]
            if len(non_zero_indices) >= 2:
                i, j = random.sample(list(non_zero_indices), 2)
                mutated[i], mutated[j] = mutated[j], mutated[i]

        elif strategy == "adjust":
            # Adjust ratio of coals (transfer between two coals)
            non_zero_indices = np.where(mutated > 0)[0]
            if len(non_zero_indices) >= 2:
                donor_idx = random.choice(non_zero_indices)
                receiver_idx = random.choice(
                    [i for i in non_zero_indices if i != donor_idx]
                )

                # Transfer 1-5 percentage points
                max_transfer = min(5, int(mutated[donor_idx] - 1))
                if max_transfer > 0:
                    transfer = (
                        random.randint(1, max_transfer) if max_transfer > 1 else 1
                    )
                    mutated[donor_idx] -= transfer
                    mutated[receiver_idx] += transfer

        elif strategy == "redistribute":
            # Redistribute within a category
            category = random.choice(list(self.category_constraints.keys()))
            category_indices = [
                i
                for i, coal in enumerate(self.coal_names)
                if self.category_map.get(coal) == category and mutated[i] > 0
            ]

            if len(category_indices) >= 2:
                total = sum(mutated[i] for i in category_indices)
                num_coals = len(category_indices)

                # Random redistribution
                new_ratios = np.random.multinomial(
                    int(total), [1 / num_coals] * num_coals
                )
                for idx, new_ratio in zip(category_indices, new_ratios):
                    mutated[idx] = new_ratio

        else:  # add_remove
            # Try to add a new coal or remove an existing one
            if random.random() < 0.5:
                # Add a new coal
                zero_indices = np.where(mutated == 0)[0]
                if len(zero_indices) > 0:
                    new_coal_idx = random.choice(zero_indices)
                    # Take from existing coals
                    non_zero_indices = np.where(mutated > 3)[
                        0
                    ]  # Only from coals with >3%
                    if len(non_zero_indices) > 0:
                        min_amount = 2
                        max_amount = 5
                        amount = (
                            random.randint(min_amount, max_amount)
                            if max_amount > min_amount
                            else min_amount
                        )
                        for _ in range(amount):
                            donor_idx = random.choice(non_zero_indices)
                            if mutated[donor_idx] > 1:
                                mutated[donor_idx] -= 1
                                mutated[new_coal_idx] += 1
            else:
                # Remove a coal (set to 0 and redistribute)
                non_zero_indices = np.where(mutated > 0)[0]
                if len(non_zero_indices) > 3:  # Keep at least 3 coals
                    remove_idx = random.choice(non_zero_indices)
                    amount = int(mutated[remove_idx])
                    mutated[remove_idx] = 0

                    # Redistribute to remaining coals
                    remaining = [i for i in non_zero_indices if i != remove_idx]
                    if remaining:
                        for _ in range(amount):
                            receiver_idx = random.choice(remaining)
                            mutated[receiver_idx] += 1

        # Ensure still sums to 100
        mutated = self.normalize_to_integer(mutated)

        return mutated
    def evolve(self):
        """Run the genetic algorithm."""
        print("Initializing population...")
        self.population = self.initialize_population()

        print(f"Evaluating initial population of {len(self.population)} blends...")
        self.fitness_scores = [
            self.evaluate_fitness(blend) for blend in self.population
        ]

        # Store population signatures with fitness
        for blend, fitness in zip(self.population, self.fitness_scores):
            sig = self.blend_to_signature(blend)
            self.population_signatures[sig] = fitness

        best_idx = np.argmin(self.fitness_scores)
        self.best_fitness = self.fitness_scores[best_idx]
        self.best_solution = self.population[best_idx].copy()

        print(f"Initial best fitness: {self.best_fitness:.2f}")
        print(f"Initial unique blends: {len(self.seen_blends)}")

        for generation in range(self.generations):
            # Sort population by fitness
            sorted_indices = np.argsort(self.fitness_scores)
            self.population = [self.population[i] for i in sorted_indices]
            self.fitness_scores = [self.fitness_scores[i] for i in sorted_indices]

            # Adaptive elite size - keep only truly unique elite
            elite_population = []
            elite_fitness = []

            for i in range(min(self.elite_size * 2, len(self.population))):
                blend = self.population[i]
                fitness = self.fitness_scores[i]

                # Check if this elite is diverse enough from already selected elites
                if (
                    self.is_diverse_enough(blend, elite_population)
                    or len(elite_population) == 0
                ):
                    elite_population.append(blend.copy())
                    elite_fitness.append(fitness)

                    if len(elite_population) >= self.elite_size:
                        break

            # Start new population with diverse elite
            new_population = elite_population
            new_fitness = elite_fitness

            # Generate offspring
            offspring_generated = 0
            attempts = 0
            max_attempts = self.population_size * 5

            while (
                len(new_population) < self.population_size and attempts < max_attempts
            ):
                attempts += 1

                # Select parents
                parent1 = self.tournament_selection()
                parent2 = self.tournament_selection()

                # Crossover
                offspring1, offspring2 = self.crossover(parent1, parent2)

                # Mutate
                offspring1 = self.mutate(offspring1)
                offspring2 = self.mutate(offspring2)

                # Repair if needed
                if not self.check_constraints(offspring1):
                    offspring1 = self.repair_blend(offspring1)
                if not self.check_constraints(offspring2):
                    offspring2 = self.repair_blend(offspring2)

                # Add to population if valid, unique, and diverse
                for offspring in [offspring1, offspring2]:
                    if len(new_population) >= self.population_size:
                        break

                    if self.check_constraints(offspring):
                        sig = self.blend_to_signature(offspring)

                        # Check if this is a new unique blend
                        if sig not in self.population_signatures:
                            # Evaluate fitness
                            fitness = self.evaluate_fitness(offspring)

                            # Check diversity
                            if (
                                self.is_diverse_enough(offspring, new_population)
                                or len(new_population) < self.elite_size * 2
                            ):
                                new_population.append(offspring)
                                new_fitness.append(fitness)
                                self.seen_blends.add(sig)
                                self.population_signatures[sig] = fitness
                                offspring_generated += 1

            # Fill remaining slots with aggressive mutations if needed
            fill_attempts = 0
            while (
                len(new_population) < self.population_size
                and fill_attempts < self.population_size * 3
            ):
                fill_attempts += 1
                max_idx = min(20, len(self.population) - 1)
                base_idx = random.randint(0, max_idx) if max_idx > 0 else 0
                base = self.population[base_idx].copy()
                mutated = self.aggressive_mutate(base)

                if self.check_constraints(mutated):
                    sig = self.blend_to_signature(mutated)
                    if sig not in self.population_signatures:
                        fitness = self.evaluate_fitness(mutated)
                        new_population.append(mutated)
                        new_fitness.append(fitness)
                        self.seen_blends.add(sig)
                        self.population_signatures[sig] = fitness

            # Update population
            self.population = new_population[: self.population_size]
            self.fitness_scores = new_fitness[: self.population_size]

            # Re-evaluate if we don't have fitness for all
            while len(self.fitness_scores) < len(self.population):
                blend = self.population[len(self.fitness_scores)]
                fitness = self.evaluate_fitness(blend)
                self.fitness_scores.append(fitness)

            # Update best solution
            best_idx = np.argmin(self.fitness_scores)
            if self.fitness_scores[best_idx] < self.best_fitness:
                self.best_fitness = self.fitness_scores[best_idx]
                self.best_solution = self.population[best_idx].copy()
                print(f"  → New best found! Fitness: {self.best_fitness:.2f}")

            # Record history
            avg_fitness = np.mean(self.fitness_scores)
            self.history.append(
                {
                    "generation": generation + 1,
                    "best_fitness": self.best_fitness,
                    "avg_fitness": avg_fitness,
                    "worst_fitness": np.max(self.fitness_scores),
                    "unique_blends": len(self.seen_blends),
                    "population_diversity": len(self.population_signatures),
                }
            )

            if (generation + 1) % 10 == 0 or generation == 0:
                print(
                    f"Gen {generation + 1}/{self.generations} - "
                    f"Best: {self.best_fitness:.2f}, Avg: {avg_fitness:.2f}, "
                    f"Unique: {len(self.population_signatures)}, New: {offspring_generated}"
                )

        print(f"\nOptimization complete!")
        print(f"Final best fitness: {self.best_fitness:.2f}")
        print(f"Total unique blends explored: {len(self.seen_blends)}")
        print(f"Unique blends in final analysis: {len(self.population_signatures)}")

    def aggressive_mutate(self, blend_array: np.ndarray) -> np.ndarray:
        """Apply multiple mutations for more diversity."""
        mutated = blend_array.copy()
        min_mutations = 2
        max_mutations = 4
        num_mutations = (
            random.randint(min_mutations, max_mutations)
            if max_mutations > min_mutations
            else min_mutations
        )

        for _ in range(num_mutations):
            mutated = self.mutate(mutated)

        return mutated


    def get_best_blend(self) -> Dict:
        """Get the best blend found."""
        if self.best_solution is None:
            return None

        blend_dict = self.blend_array_to_dict(self.best_solution)
        model_result = self.predictor.predict(blend_dict, verbose=False)
        blend_cost = self.predictor.Blend_cost(blend_dict)

        return {
            "blend": blend_dict,
            "fitness": self.best_fitness,
            "predictions": model_result["predictions"],
            "category_distribution": model_result["category_distribution"],
            "blend_cost": blend_cost["Total blend Cost"],
        }

    def get_top_n_blends(self, n: int = 5, min_diversity: float = None) -> List[Dict]:
        """
        Get top N best unique and diverse blends.

        Args:
            n: Number of top blends to return
            min_diversity: Minimum diversity required between returned blends (defaults to self.diversity_threshold)
        """
        if min_diversity is None:
            min_diversity = self.diversity_threshold

        # Get all unique blends with their fitness
        unique_blends = []
        seen_sigs = set()

        sorted_indices = np.argsort(self.fitness_scores)

        for idx in sorted_indices:
            sig = self.blend_to_signature(self.population[idx])
            if sig not in seen_sigs:
                unique_blends.append(idx)
                seen_sigs.add(sig)

        # Now select diverse blends from the unique ones
        diverse_blends = []
        diverse_blend_arrays = []

        for idx in unique_blends:
            blend = self.population[idx]

            # Check if this blend is diverse enough from already selected blends
            is_diverse = True
            for selected_blend in diverse_blend_arrays:
                diversity = self.calculate_diversity(blend, selected_blend)
                if diversity < min_diversity:
                    is_diverse = False
                    break

            if is_diverse or len(diverse_blends) == 0:
                diverse_blends.append(idx)
                diverse_blend_arrays.append(blend)

            if len(diverse_blends) >= n:
                break

        # Build result list
        top_blends = []
        for idx in diverse_blends:
            blend_dict = self.blend_array_to_dict(self.population[idx])
            model_result = self.predictor.predict(blend_dict, verbose=False)
            blend_cost = self.predictor.Blend_cost(blend_dict)

            top_blends.append(
                {
                    "blend": blend_dict,
                    "fitness": self.fitness_scores[idx],
                    "predictions": model_result["predictions"],
                    "category_distribution": model_result["category_distribution"],
                    "blend_cost": blend_cost["Total blend Cost"],
                }
            )

        return top_blends


# Example usage:
if __name__ == "__main__":
    predictor = CoalBlendPredictor()

    ga = CoalBlendGA(
        best_blend_csv="Best_blend_ratio.csv",
        clean_exist_coals_csv="Clean_exist_coals.csv",
        predictor=predictor,
        population_size=150,
        generations=75,
    )
    print(CoalBlendGA)
    print(dir(CoalBlendGA))

    # Run optimization
    ga.evolve()

    # Get best blend
    best_blend = ga.get_best_blend()
    print("\nBest Blend Found:")
    print(f"Blend: {best_blend['blend']}")
    print(f"Predictions: {best_blend['predictions']}")
    print(f"Cost: {best_blend['blend_cost']}")

    # Get top 5 blends
    top_blends = ga.get_top_n_blends(5)
    for i, blend in enumerate(top_blends, 1):
        print(f"\nTop {i} Blend:")
        print(f"Fitness: {blend['fitness']:.2f}")
        print(f"Predictions: {blend['predictions']}")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# import pandas as pd
# import numpy as np
# import random
# from typing import Dict, List, Tuple
# from copy import deepcopy
# import pandas as pd
# import numpy as np
# import pickle
# import re
# from pathlib import Path
# from pycaret.regression import load_model, predict_model


# class CoalBlendPredictor:
#     """
#     Coal Blend Prediction System

#     Predicts CRI, CSR, ASH, and VM values based on coal blend ratios.
#     """

#     def __init__(self, coal_db_path="Clean_exist_coals.csv", models_dir="."):
#         """
#         Initialize the Coal Blend Predictor

#         Parameters:
#         -----------
#         coal_db_path : str
#             Path to Clean_exist_coals.csv
#         scaler_path : str
#             Path to scaler_colwise.pkl
#         models_dir : str
#             Directory containing model files (CRI_model.pkl, etc.)
#         """
#         self.coal_db_path = coal_db_path
#         self.models_dir = Path(models_dir)

#         # Initialize coal database
#         self.coal_db = None
#         self.feature_names = None
#         self.models = {}
#         self.model_features = {}

#         # Load all necessary files
#         self._load_resources()

#     def _load_resources(self):
#         """Load coal database, scaler, and models"""
#         try:
#             # Load coal database
#             self.coal_db = self._load_coal_database()
#             self.coal_db = self._feature_engineering(self.coal_db)
#             self.coal_db = self._prepare_coal_properties(self.coal_db)

#             # Load scaler
#             # with open(self.scaler_path, "rb") as f:
#             #     scaler_data = pickle.load(f)
#             # self.scaler = scaler_data['scaler']
#             # self.feature_names = scaler_data['feature_names']

#             # Load models
#             targets = ["CRI", "CSR", "ASH", "VM"]
#             for target in targets:
#                 model_path = self.models_dir / f"{target}_model"
#                 features_path = self.models_dir / f"{target}_features.pkl"

#                 self.models[target] = load_model(str(model_path))

#                 with open(features_path, "rb") as f:
#                     self.model_features[target] = pickle.load(f)

#         except Exception as e:
#             raise RuntimeError(f"Failed to load resources: {str(e)}")

#     @staticmethod
#     def _normalize_name(s):
#         """Normalize coal names for matching"""
#         s = str(s).lower().strip()
#         s = re.sub(r"[^a-z0-9]+", " ", s)
#         s = re.sub(r"\s+", " ", s).strip()
#         return s

#     def _load_coal_database(self):
#         """Load and prepare coal properties database"""
#         df = pd.read_csv(self.coal_db_path)

#         # Clean column names
#         df.columns = df.columns.str.strip()
#         df.columns = df.columns.str.replace(r"\s+", " ", regex=True)
#         df.columns = df.columns.str.replace("\ufeff", "")

#         # Fill missing values
#         df = df.fillna(0)

#         # Create normalized name column for matching
#         df["__norm_name"] = df["Name_of_coal"].map(self._normalize_name)

#         return df

#     def _feature_engineering(self, df):
#         """Apply feature engineering (BI, MBI, VR, CBI, Log_Max_Fluidity)"""

#         # 1. Calculate BI
#         df["BI"] = (df["Fe2O3"] + df["CaO"] + df["MgO"] + df["Na2O"] + df["K2O"]) / (
#             df["SiO2"] + df["Al2O3"] + df["TiO2"]
#         )

#         # 2. Calculate MBI
#         df["MBI"] = ((df["Ash"] * 100) / (100 - df["V.M."])) * df["BI"]

#         # 3. VR7 to VR19
#         ratios = {
#             "V7": 3,
#             "V8": 2.7,
#             "V9": 2.5,
#             "V10": 2.4,
#             "V11": 2.5,
#             "V12": 3,
#             "V13": 3.7,
#             "V14": 5,
#             "V15": 7,
#             "V16": 9.6,
#             "V17": 12,
#             "V18": 15,
#             "V19": 18,
#         }

#         for col, divisor in ratios.items():
#             if col in df.columns:
#                 df[f"VR{col[1:]}"] = df[col] / divisor

#         # 4. Calculate CBI only for HCC, HFCC, SHCC
#         VR_cols = [f"VR{i}" for i in range(7, 20) if f"VR{i}" in df.columns]
#         sum_vr = df[VR_cols].sum(axis=1)

#         df["CBI"] = np.where(
#             (df["Coal Category"].isin(["HCC", "HFCC", "SHCC"])) & (sum_vr != 0),
#             (df["Inertinite"] + df["Minerals"]) / sum_vr,
#             0,
#         )

#         # 5. Log of Max. Fluidity ddpm
#         if "Max. Fluidity ddpm" in df.columns:
#             df["Log_Max_Fluidity"] = np.log(
#                 df["Max. Fluidity ddpm"].replace(0, np.nan)
#             ).fillna(0)

#         return df

#     def _prepare_coal_properties(self, df):
#         """Prepare coal properties after feature engineering"""

#         # Columns to drop
#         cols_to_drop = (
#             ["Max. Fluidity ddpm"]
#             + [f"V{i}" for i in range(7, 20)]
#             + [f"VR{i}" for i in range(7, 20)]
#             + [
#                 "Inertinite",
#                 "Minerals",
#                 "SiO2",
#                 "Al2O3",
#                 "Fe2O3",
#                 "CaO",
#                 "MgO",
#                 "Na2O",
#                 "K2O",
#                 "TiO2",
#                 "Mn3O4",
#                 "SO3",
#                 "P2O5",
#                 "BaO",
#                 "SrO",
#                 "ZnO",
#             ]
#             + ["C", "H", "O", "N", "S"]
#             + [
#                 "HGI (ASTM)",
#                 "Rank",
#                 "Vitrinite %",
#                 "Liptinite",
#                 "Semi-Fusinite",
#                 "Initial Softening Temp.",
#             ]
#         )

#         # Drop only existing columns
#         df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

#         return df

#     def _calculate_weighted_features(self, coal_inputs):
#         """
#         Calculate weighted features based on coal inputs

#         Parameters:
#         -----------
#         coal_inputs : dict
#             Dictionary like {'Goonyella': 50, 'R.PCI': 50}

#         Returns:
#         --------
#         dict : Category percentages and weighted features
#         """

#         # Initialize category percentages
#         categories = {"HCC": 0, "SHCC": 0, "HFCC": 0, "PCI": 0, "WC": 0}

#         # Property columns (exclude Name and Category)
#         exclude_cols = ["Name_of_coal", "__norm_name", "Coal Category"]
#         prop_cols = [c for c in self.coal_db.columns if c not in exclude_cols]

#         # Initialize weighted sums
#         weighted_features = {f"weighted_{col}": 0 for col in prop_cols}

#         # Process each coal input
#         for coal_name, ratio in coal_inputs.items():
#             norm_name = self._normalize_name(coal_name)

#             # Find coal in database
#             coal_row = self.coal_db[self.coal_db["__norm_name"] == norm_name]

#             if coal_row.empty:
#                 raise ValueError(f"Coal '{coal_name}' not found in database!")

#             coal_row = coal_row.iloc[0]

#             # Add to category
#             category = coal_row["Coal Category"]
#             if category in categories:
#                 categories[category] += ratio

#             # Calculate weighted properties
#             for col in prop_cols:
#                 weighted_features[f"weighted_{col}"] += coal_row[col] * ratio

#         # Combine categories and weighted features
#         result = {**categories, **weighted_features}

#         return result

#     # def _calculate_cri_csr(self, blend_features):
#     #     """Calculate CRI and CSR using formulas"""

#     #     vm = blend_features['weighted_V.M.']
#     #     log_fluidity = blend_features['weighted_Log_Max_Fluidity']

#     #     # Calculate CRI
#     #     cri_direct = -16.48 + 8.16 * vm - 21.68 * log_fluidity

#     #     # Calculate CSR (two formulas)
#     #     csr_from_cri = 94.19 - 1.15 * cri_direct
#     #     csr_direct = 95.76 - 2.50 * vm + 11.00 * log_fluidity

#     #     blend_features['CRI_direct'] = cri_direct
#     #     blend_features['CSR_from_CRI'] = csr_from_cri
#     #     blend_features['CSR_direct'] = csr_direct

#     #     return blend_features

#     # def _normalize_features(self, blend_features):
#     #     """Normalize features using loaded scaler"""

#     #     # Extract feature values in correct order
#     #     feature_values = [blend_features[fname] for fname in self.feature_names]
#     #     feature_array = np.array(feature_values).reshape(1, -1)

#     #     # Normalize (column-wise)
#     #     normalized = self.scaler.transform(feature_array)

#     #     # Create normalized dict
#     #     normalized_dict = dict(zip(self.feature_names, normalized[0]))

#     #     return normalized_dict

#     def _predict_targets(self, normalized_features):
#         """Predict CRI, CSR, ASH, VM using loaded models"""

#         predictions = {}

#         for target in ["CRI", "CSR", "ASH", "VM"]:
#             # Prepare input DataFrame
#             input_df = pd.DataFrame([normalized_features])
#             input_df = input_df[self.model_features[target]]

#             # Predict
#             pred = predict_model(self.models[target], data=input_df)
#             predictions[target] = pred["prediction_label"].iloc[0]

#         return predictions

#     def predict(self, coal_inputs, verbose=False):
#         """
#         Main prediction method

#         Parameters:
#         -----------
#         coal_inputs : dict
#             Dictionary of coal names and their ratios
#             Example: {'Goonyella': 50, 'R.PCI': 50}
#         verbose : bool
#             If True, print detailed information

#         Returns:
#         --------
#         dict : Contains blend_composition, category_distribution, and predictions

#         Example:
#         --------
#         >>> predictor = CoalBlendPredictor()
#         >>> result = predictor.predict({'Goonyella': 50, 'R.PCI': 50})
#         >>> print(result['predictions']['CRI'])
#         27.5
#         """

#         if verbose:
#             print("🔮 Starting prediction...")

#         # Validate total ratio
#         total_ratio = sum(coal_inputs.values())
#         if abs(total_ratio - 100) > 0.01:
#             raise ValueError(f"Total ratio must be 100%, got {total_ratio}%")

#         try:
#             # Calculate weighted features
#             blend_features = self._calculate_weighted_features(coal_inputs)

#             # Calculate CRI/CSR
#             # blend_features = self._calculate_cri_csr(blend_features)

#             # # Normalize features
#             # normalized_features = self._normalize_features(blend_features)

#             # Predict targets
#             predictions = self._predict_targets(blend_features)

#             # Prepare result
#             result = {
#                 "blend_composition": coal_inputs,
#                 "category_distribution": {
#                     k: blend_features[k] for k in ["HCC", "SHCC", "HFCC", "PCI", "WC"]
#                 },
#                 "predictions": predictions,
#                 "status": "success",
#             }

#             if verbose:
#                 self._print_results(result)
#             print(result)
#             return result

#         except Exception as e:
#             return {
#                 "status": "error",
#                 "error_message": str(e),
#                 "blend_composition": coal_inputs,
#             }

#     def _print_results(self, result):
#         """Print formatted results"""
#         print("\n" + "=" * 60)
#         print("BLEND COMPOSITION")
#         print("=" * 60)
#         for coal, ratio in result["blend_composition"].items():
#             print(f"  {coal}: {ratio}%")

#         print("\n" + "=" * 60)
#         print("COAL CATEGORY DISTRIBUTION")
#         print("=" * 60)
#         for cat, value in result["category_distribution"].items():
#             print(f"  {cat}: {value}%")

#         print("\n" + "=" * 60)
#         print("PREDICTIONS")
#         print("=" * 60)
#         for target, value in result["predictions"].items():
#             print(f"  {target}: {value:.2f}")
#         print("=" * 60 + "\n")

#     def batch_predict(self, coal_inputs_list, verbose=False):
#         """
#         Predict for multiple blends

#         Parameters:
#         -----------
#         coal_inputs_list : list of dict
#             List of coal input dictionaries
#         verbose : bool
#             If True, print progress

#         Returns:
#         --------
#         list : List of prediction results
#         """
#         results = []

#         for i, coal_inputs in enumerate(coal_inputs_list):
#             if verbose:
#                 print(f"\n[{i+1}/{len(coal_inputs_list)}] Processing blend...")

#             result = self.predict(coal_inputs, verbose=verbose)
#             results.append(result)

#         return results

#     def get_available_coals(self):
#         """
#         Get list of available coal names in the database

#         Returns:
#         --------
#         list : List of coal names
#         """
#         return self.coal_db["Name_of_coal"].tolist()

#     def get_coal_info(self, coal_name):
#         """
#         Get information about a specific coal

#         Parameters:
#         -----------
#         coal_name : str
#             Name of the coal

#         Returns:
#         --------
#         dict : Coal properties and category
#         """
#         norm_name = self._normalize_name(coal_name)
#         coal_row = self.coal_db[self.coal_db["__norm_name"] == norm_name]

#         if coal_row.empty:
#             return None

#         coal_row = coal_row.iloc[0]
#         return {
#             "name": coal_row["Name_of_coal"],
#             "category": coal_row["Coal Category"],
#             "properties": coal_row.drop(
#                 ["Name_of_coal", "__norm_name", "Coal Category"]
#             ).to_dict(),
#         }

#     def Blend_cost(self, blend_dict):
#         # Load coal cost data
#         coal_costs = pd.read_csv("/content/coal_costs.csv")
#         coal_costs.set_index("Name_of_coal", inplace=True)
#         total_cost = 0.0
#         for coal_name, percentage in blend_dict.items():
#             if coal_name not in coal_costs.index:
#                 raise ValueError(f"Coal '{coal_name}' not found in coal_costs.csv")
#             cost_per_ton = coal_costs.loc[coal_name, "Cost"]
#             ton_fraction = percentage / 100  # convert % to fraction
#             total_cost += cost_per_ton * ton_fraction
#         return {"Total blend Cost": float(total_cost)}

#     """
#             Below is the way to make input for cost function i.e cost_input
#             predictions = model_result.get('predictions', {})
#             cost_dict = predictor.Blend_cost(blend)
#             cost_fn_input = {'Total blend Cost': cost_dict.get('Total blend Cost', 0.0), 'predictions': predictions}
#             cost_result = predictor.cost_function(cost_fn_input)
#     """

#     def cost_function(self, cost_input):
#         # combine difference from desired values + blend cost in a simple way
#         desired = {
#             "CRI": {"type": "range", "min": 22, "max": 28, "weight": 1.0},
#             "CSR": {"type": "min", "min": 63, "weight": 1.5},
#             "ASH": {"type": "max", "max": 13.5, "weight": 1.2},
#         }
#         preds = cost_input["predictions"]
#         penalty = 0.0
#         # CRI: penalty outside range
#         d = desired["CRI"]
#         if preds["CRI"] < d["min"]:
#             penalty += (d["min"] - preds["CRI"]) * d["weight"] * 1000
#         elif preds["CRI"] > d["max"]:
#             penalty += (preds["CRI"] - d["max"]) * d["weight"] * 1000
#         # CSR: penalty if below min
#         d = desired["CSR"]
#         if preds["CSR"] < d["min"]:
#             penalty += (d["min"] - preds["CSR"]) * d["weight"] * 1000
#         # ASH: penalty if above max
#         d = desired["ASH"]
#         if preds["ASH"] > d["max"]:
#             penalty += (preds["ASH"] - d["max"]) * d["weight"] * 1000
#         # plus blend cost influence
#         total_cost = cost_input.get("Total blend Cost", 0)
#         # combine
#         score = total_cost + penalty
#         return {"Cost function evaluation": float(score)}

#     def predict_from_csv(self, input_csv_path, output_csv_path=None, verbose=True):
#         """
#         Predict from CSV file where columns are coal names and values are ratios

#         Parameters:
#         -----------
#         input_csv_path : str
#             Path to input CSV file with coal names as columns and ratios as values
#         output_csv_path : str, optional
#             Path to save output CSV with predictions appended
#             If None, will add '_predictions' to input filename
#         verbose : bool
#             If True, print progress

#         Returns:
#         --------
#         pd.DataFrame : DataFrame with original data + predictions

#         Example CSV format:
#         Goonyella,R.PCI,Peak Downs,CRI,CSR,ASH,VM
#         50,30,20,,,,
#         40,40,20,,,,
#         """

#         if verbose:
#             print(f"\n{'='*60}")
#             print("COAL BLEND CSV PREDICTION")
#             print(f"{'='*60}")
#             print(f"📂 Loading: {input_csv_path}")

#         # Load CSV
#         df = pd.read_csv(input_csv_path)

#         # Clean column names
#         df.columns = df.columns.str.strip()

#         # Identify prediction columns (CRI, CSR, ASH, VM) if they exist
#         prediction_cols = ["CRI", "CSR", "ASH", "VM"]
#         existing_pred_cols = [col for col in prediction_cols if col in df.columns]

#         # Get coal columns (all columns except prediction columns)
#         coal_columns = [col for col in df.columns if col not in prediction_cols]

#         if verbose:
#             print(f"🔍 Found {len(coal_columns)} coal columns")
#             print(f"📊 Processing {len(df)} blend(s)...")

#         # Initialize prediction columns if they don't exist
#         for col in prediction_cols:
#             if col not in df.columns:
#                 df[col] = np.nan

#         # Process each row
#         successful = 0
#         failed = 0
#         errors = []

#         for idx, row in df.iterrows():
#             # Create coal input dictionary (only non-zero/non-empty values)
#             coal_inputs = {}
#             for coal_name in coal_columns:
#                 value = row[coal_name]
#                 # Handle empty strings, NaN, and zeros
#                 if pd.notna(value) and value != "" and float(value) != 0:
#                     coal_inputs[coal_name] = float(value)

#             # Skip if no valid coal inputs
#             if not coal_inputs:
#                 if verbose:
#                     print(f"  ⚠️  Row {idx+1}: No valid coal ratios found, skipping...")
#                 failed += 1
#                 errors.append(f"Row {idx+1}: No valid coal ratios")
#                 continue

#             # Predict
#             result = self.predict(coal_inputs, verbose=False)

#             if result["status"] == "success":
#                 # Update predictions in DataFrame
#                 for target in prediction_cols:
#                     df.at[idx, target] = result["predictions"][target]
#                 successful += 1

#                 if verbose:
#                     blend_str = ", ".join([f"{k}:{v}%" for k, v in coal_inputs.items()])
#                     print(f"  ✅ Row {idx+1}: {blend_str}")
#                     print(
#                         f"      → CRI={result['predictions']['CRI']:.2f}, "
#                         f"CSR={result['predictions']['CSR']:.2f}, "
#                         f"ASH={result['predictions']['ASH']:.2f}, "
#                         f"VM={result['predictions']['VM']:.2f}"
#                     )
#             else:
#                 failed += 1
#                 errors.append(f"Row {idx+1}: {result['error_message']}")
#                 if verbose:
#                     print(f"  ❌ Row {idx+1}: {result['error_message']}")

#         # Save output
#         if output_csv_path is None:
#             # Generate output filename
#             input_path = Path(input_csv_path)
#             output_csv_path = (
#                 input_path.parent / f"{input_path.stem}_predictions{input_path.suffix}"
#             )

#         df.to_csv(output_csv_path, index=False)

#         if verbose:
#             print(f"\n{'='*60}")
#             print("SUMMARY")
#             print(f"{'='*60}")
#             print(f"✅ Successful predictions: {successful}/{len(df)}")
#             if failed > 0:
#                 print(f"❌ Failed predictions: {failed}/{len(df)}")
#                 print("\nErrors:")
#                 for error in errors:
#                     print(f"  • {error}")
#             print(f"\n💾 Output saved to: {output_csv_path}")
#             print(f"{'='*60}\n")

#         return df


# class CoalBlendGA:
#     """
#     Genetic Algorithm for optimizing coal blend ratios with integer-based ratios.
#     """

#     def __init__(
#         self,
#         best_blend_csv: str,
#         clean_exist_coals_csv: str,
#         predictor,
#         population_size: int = 150,
#         generations: int = 75,
#         crossover_rate: float = 0.8,
#         mutation_rate: float = 0.3,
#         elite_size: int = 5,
#         use_integer_ratios: bool = True,
#         diversity_threshold: float = 0.15,
#     ):
#         """
#         Initialize the Genetic Algorithm.

#         Args:
#             best_blend_csv: Path to Best_blend_ratio.csv
#             clean_exist_coals_csv: Path to Clean_exist_coals.csv
#             predictor: Model predictor object with predict(), Blend_cost(), and cost_function() methods
#             population_size: Number of individuals in population
#             generations: Number of generations to evolve
#             crossover_rate: Probability of crossover
#             mutation_rate: Probability of mutation
#             elite_size: Number of top individuals to preserve
#             use_integer_ratios: If True, uses integer percentages (recommended)
#             diversity_threshold: Minimum difference required between blends (0-1)
#         """
#         self.predictor = predictor
#         self.population_size = population_size
#         self.generations = generations
#         self.crossover_rate = crossover_rate
#         self.mutation_rate = mutation_rate
#         self.elite_size = elite_size
#         self.use_integer_ratios = use_integer_ratios
#         self.diversity_threshold = diversity_threshold

#         # Load data
#         self.initial_blends = pd.read_csv(best_blend_csv)
#         self.coal_categories = pd.read_csv(clean_exist_coals_csv)

#         # Extract coal names
#         self.coal_names = list(self.initial_blends.columns)

#         # Create category mapping
#         self.category_map = dict(
#             zip(
#                 self.coal_categories["Name_of_coal"],
#                 self.coal_categories["Coal Category"],
#             )
#         )

#         # Define category constraints
#         self.category_constraints = {
#             "SHCC": {"highest_count": 3, "min_ratio": 10, "max_ratio": 52},
#             "HCC": {"highest_count": 2, "min_ratio": 24, "max_ratio": 59},
#             "HFCC": {"highest_count": 2, "min_ratio": 3, "max_ratio": 25},
#             "WC": {"highest_count": 2, "min_ratio": 1, "max_ratio": 15},
#             "PCI": {"highest_count": 1, "min_ratio": 2, "max_ratio": 28},
#         }

#         # Desired values for optimization
#         self.desired_values = {
#             "CRI": {"type": "range", "min": 22, "max": 28, "weight": 1.0},
#             "CSR": {"type": "min", "min": 63, "weight": 1.5},
#             "ASH": {"type": "max", "max": 13.5, "weight": 1.2},
#         }

#         self.population = []
#         self.fitness_scores = []
#         self.best_solution = None
#         self.best_fitness = float("inf")
#         self.history = []
#         self.seen_blends = set()  # Track unique blends to avoid duplicates
#         self.population_signatures = {}  # Map signature to fitness for quick lookup

#     def calculate_diversity(self, blend1: np.ndarray, blend2: np.ndarray) -> float:
#         """
#         Calculate diversity between two blends (0 = identical, 1 = completely different).
#         Uses normalized Manhattan distance.
#         """
#         blend1_norm = blend1 / 100.0  # Normalize to 0-1
#         blend2_norm = blend2 / 100.0

#         # Manhattan distance normalized by maximum possible distance
#         distance = np.sum(np.abs(blend1_norm - blend2_norm)) / 2.0
#         return distance

#     def is_diverse_enough(
#         self, blend: np.ndarray, population: List[np.ndarray]
#     ) -> bool:
#         """Check if blend is sufficiently different from existing population."""
#         if len(population) == 0:
#             return True

#         for existing_blend in population:
#             diversity = self.calculate_diversity(blend, existing_blend)
#             if diversity < self.diversity_threshold:
#                 return False

#         return True

#     def normalize_to_integer(self, blend_array: np.ndarray) -> np.ndarray:
#         """
#         Normalize blend to sum to 100 using integer values.
#         This eliminates decimal precision issues.
#         """
#         # Round to integers
#         blend_int = np.round(blend_array).astype(int)

#         # Ensure sum is exactly 100
#         current_sum = np.sum(blend_int)
#         diff = 100 - current_sum

#         if diff != 0:
#             # Find non-zero indices to adjust
#             non_zero_indices = np.where(blend_int > 0)[0]

#             if len(non_zero_indices) > 0:
#                 # Distribute the difference
#                 if diff > 0:
#                     # Need to add
#                     for _ in range(abs(diff)):
#                         idx = random.choice(non_zero_indices)
#                         blend_int[idx] += 1
#                 else:
#                     # Need to subtract
#                     for _ in range(abs(diff)):
#                         # Only subtract from coals with ratio > 1
#                         valid_indices = [
#                             i for i in non_zero_indices if blend_int[i] > 1
#                         ]
#                         if valid_indices:
#                             idx = random.choice(valid_indices)
#                             blend_int[idx] -= 1
#                         else:
#                             break

#         return blend_int.astype(float)

#     def blend_to_signature(self, blend_array: np.ndarray) -> str:
#         """Create a unique signature for a blend to detect duplicates."""
#         blend_int = np.round(blend_array).astype(int)
#         return ",".join(map(str, blend_int))

#     def blend_array_to_dict(self, blend_array: np.ndarray) -> Dict[str, float]:
#         """Convert blend array to dictionary format."""
#         if self.use_integer_ratios:
#             blend_array = np.round(blend_array).astype(int)
#         return {
#             coal: float(ratio)
#             for coal, ratio in zip(self.coal_names, blend_array)
#             if ratio > 0
#         }

#     def blend_dict_to_array(self, blend_dict: Dict[str, float]) -> np.ndarray:
#         """Convert blend dictionary to array format."""
#         array = np.zeros(len(self.coal_names))
#         for i, coal in enumerate(self.coal_names):
#             array[i] = blend_dict.get(coal, 0)
#         if self.use_integer_ratios:
#             array = self.normalize_to_integer(array)
#         return array

#     def check_constraints(
#         self, blend_array: np.ndarray, check_soft: bool = True
#     ) -> bool:
#         """
#         Check if blend satisfies all constraints.

#         Args:
#             blend_array: Array of coal blend ratios
#             check_soft: If True, also check soft constraints
#         """
#         if self.use_integer_ratios:
#             blend_array = np.round(blend_array).astype(int)

#         # Check sum equals 100
#         if abs(np.sum(blend_array) - 100) > 0.5:
#             return False

#         # Calculate weighted properties
#         properties = {}
#         for prop in ["PCI", "WC", "HCC", "SHCC", "HFCC"]:
#             if hasattr(self, f"{prop.lower()}_values"):
#                 values = getattr(self, f"{prop.lower()}_values")
#                 properties[prop] = np.sum(blend_array * values) / 100

#         # Hard constraints
#         if "PCI" in properties and properties["PCI"] > 10:
#             return False

#         if "WC" in properties and properties["WC"] > 8:
#             return False

#         if "HCC" in properties and properties["HCC"] < 30:
#             return False

#         if "SHCC" in properties and properties["SHCC"] < 20:
#             return False

#         # Soft constraints (optional check)
#         if check_soft:
#             if "HCC" in properties and "SHCC" in properties:
#                 if properties["HCC"] + properties["SHCC"] < 60:
#                     return False

#             if "PCI" in properties and "WC" in properties:
#                 if properties["PCI"] + properties["WC"] > 15:
#                     return False

#             if "SHCC" in properties and "HFCC" in properties:
#                 if properties["SHCC"] < properties["HFCC"]:
#                     return False

#         # Check category constraints
#         category_ratios = {}
#         category_counts = {}

#         for i, coal in enumerate(self.coal_names):
#             if blend_array[i] > 0:
#                 category = self.category_map.get(coal, "Unknown")
#                 if category == "Unknown":
#                     continue

#                 category_ratios[category] = (
#                     category_ratios.get(category, 0) + blend_array[i]
#                 )
#                 category_counts[category] = category_counts.get(category, 0) + 1

#         # Validate each category
#         for category, constraints in self.category_constraints.items():
#             ratio = category_ratios.get(category, 0)
#             count = category_counts.get(category, 0)

#             # If category is used, check constraints
#             if ratio > 0:
#                 if count > constraints["highest_count"]:
#                     return False
#                 if ratio < constraints["min_ratio"] or ratio > constraints["max_ratio"]:
#                     return False

#         return True

#     def generate_random_blend(
#         self, max_attempts: int = 1000, prioritize_soft: bool = True
#     ) -> np.ndarray:
#         """
#         Generate a random blend with integer ratios satisfying constraints.

#         Args:
#             max_attempts: Maximum number of attempts to generate valid blend
#             prioritize_soft: If True, try to satisfy soft constraints
#         """
#         for attempt in range(max_attempts):
#             blend_array = np.zeros(len(self.coal_names))

#             # Target ratios for each category
#             category_targets = {}

#             for category, constraints in self.category_constraints.items():
#                 category_coals = [
#                     coal
#                     for coal in self.coal_names
#                     if self.category_map.get(coal) == category
#                 ]

#                 if not category_coals:
#                     continue

#                 # Randomly decide how many coals to use
#                 max_coals = min(constraints["highest_count"], len(category_coals))
#                 num_coals = random.randint(1, max_coals) if max_coals > 1 else 1
#                 selected_coals = random.sample(category_coals, num_coals)

#                 # Random ratio within constraints (integer)
#                 min_ratio = constraints["min_ratio"]
#                 max_ratio = constraints["max_ratio"]
#                 category_ratio = (
#                     random.randint(min_ratio, max_ratio)
#                     if max_ratio > min_ratio
#                     else min_ratio
#                 )
#                 category_targets[category] = (selected_coals, category_ratio)

#             # Distribute ratios
#             total_target = sum(target[1] for target in category_targets.values())

#             if total_target > 0:
#                 # Scale to approximately 100
#                 scale_factor = 100 / total_target

#                 for category, (coals, ratio) in category_targets.items():
#                     scaled_ratio = int(round(ratio * scale_factor))

#                     # Distribute among coals in category
#                     num_coals = len(coals)
#                     base_ratio = scaled_ratio // num_coals
#                     remainder = scaled_ratio % num_coals

#                     for i, coal in enumerate(coals):
#                         idx = self.coal_names.index(coal)
#                         blend_array[idx] = base_ratio + (1 if i < remainder else 0)

#             # Normalize to exactly 100
#             blend_array = self.normalize_to_integer(blend_array)

#             # Check hard constraints (and optionally soft)
#             if self.check_constraints(blend_array, check_soft=False):
#                 # If we want soft constraints, check those too
#                 if prioritize_soft:
#                     if self.check_constraints(blend_array, check_soft=True):
#                         return blend_array
#                     # Continue trying if soft constraints not met
#                 else:
#                     return blend_array

#         # If max_attempts reached and prioritize_soft was True,
#         # try once more with only hard constraints
#         if prioritize_soft:
#             for attempt in range(max_attempts):
#                 blend_array = self._generate_blend_once()
#                 if self.check_constraints(blend_array, check_soft=False):
#                     return blend_array

#         # Return best effort blend
#         return self.normalize_to_integer(np.zeros(len(self.coal_names)))


#     # def _generate_blend_once(self) -> np.ndarray:
#     #     """Helper method to generate a single random blend."""
#     #     blend_array = np.zeros(len(self.coal_names))
#     #     category_targets = {}

#     #     for category, constraints in self.category_constraints.items():
#     #         category_coals = [
#     #             coal for coal in self.coal_names if self.category_map.get(coal) == category
#     #         ]

#     #         if not category_coals:
#     #             continue

#     #         max_coals = min(constraints["highest_count"], len(category_coals))
#     #         num_coals = random.randint(1, max_coals) if max_coals > 1 else 1
#     #         selected_coals = random.sample(category_coals, num_coals)

#     #         min_ratio = constraints["min_ratio"]
#     #         max_ratio = constraints["max_ratio"]
#     #         category_ratio = (
#     #             random.randint(min_ratio, max_ratio) if max_ratio > min_ratio else min_ratio
#     #         )
#     #         category_targets[category] = (selected_coals, category_ratio)

#     #     total_target = sum(target[1] for target in category_targets.values())

#     #     if total_target > 0:
#     #         scale_factor = 100 / total_target

#     #         for category, (coals, ratio) in category_targets.items():
#     #             scaled_ratio = int(round(ratio * scale_factor))
#     #             num_coals = len(coals)
#     #             base_ratio = scaled_ratio // num_coals
#     #             remainder = scaled_ratio % num_coals

#     #             for i, coal in enumerate(coals):
#     #                 idx = self.coal_names.index(coal)
#     #                 blend_array[idx] = base_ratio + (1 if i < remainder else 0)

#     #     return self.normalize_to_integer(blend_array)
#     def repair_blend(self, blend_array: np.ndarray, max_iterations: int = 100) -> np.ndarray:
#         """
#         Repair a blend to satisfy all constraints.

#         Args:
#             blend_array: Array of coal blend ratios to repair
#             max_iterations: Maximum number of repair attempts

#         Returns:
#             Repaired blend array
#         """
#         if self.use_integer_ratios:
#             blend_array = np.round(blend_array).astype(int)

#         for iteration in range(max_iterations):
#             # First normalize to sum = 100
#             blend_array = self.normalize_to_integer(blend_array)

#             # Check if already valid
#             if self.check_constraints(blend_array):
#                 return blend_array

#             # Calculate current properties
#             properties = {}
#             for prop in ['PCI', 'WC', 'HCC', 'SHCC', 'HFCC']:
#                 prop_attr = f'{prop.lower()}_values'
#                 if hasattr(self, prop_attr):
#                     values = getattr(self, prop_attr)
#                     properties[prop] = np.sum(blend_array * values) / 100

#             # Repair hard constraints by adjusting blend
#             adjusted = False

#             # Fix PCI > 10: reduce high-PCI coals
#             if 'PCI' in properties and properties['PCI'] > 10:
#                 pci_values = self.pci_values
#                 high_pci_indices = np.where((blend_array > 0) & (pci_values > properties['PCI']))[0]
#                 if len(high_pci_indices) > 0:
#                     idx = np.random.choice(high_pci_indices)
#                     blend_array[idx] = max(0, blend_array[idx] - 1)
#                     adjusted = True

#             # Fix WC > 8: reduce high-WC coals
#             if 'WC' in properties and properties['WC'] > 8:
#                 wc_values = self.wc_values
#                 high_wc_indices = np.where((blend_array > 0) & (wc_values > properties['WC']))[0]
#                 if len(high_wc_indices) > 0:
#                     idx = np.random.choice(high_wc_indices)
#                     blend_array[idx] = max(0, blend_array[idx] - 1)
#                     adjusted = True

#             # Fix HCC < 30: increase high-HCC coals
#             if 'HCC' in properties and properties['HCC'] < 30:
#                 hcc_values = self.hcc_values
#                 high_hcc_indices = np.where(hcc_values > properties['HCC'])[0]
#                 if len(high_hcc_indices) > 0:
#                     idx = np.random.choice(high_hcc_indices)
#                     blend_array[idx] = blend_array[idx] + 1
#                     adjusted = True

#             # Fix SHCC < 20: increase high-SHCC coals
#             if 'SHCC' in properties and properties['SHCC'] < 20:
#                 shcc_values = self.shcc_values
#                 high_shcc_indices = np.where(shcc_values > properties['SHCC'])[0]
#                 if len(high_shcc_indices) > 0:
#                     idx = np.random.choice(high_shcc_indices)
#                     blend_array[idx] = blend_array[idx] + 1
#                     adjusted = True

#             # Fix category constraints
#             category_ratios = {}
#             category_counts = {}
#             category_coals = {}

#             for i, coal in enumerate(self.coal_names):
#                 if blend_array[i] > 0:
#                     category = self.category_map.get(coal, 'Unknown')
#                     if category != 'Unknown':
#                         category_ratios[category] = category_ratios.get(category, 0) + blend_array[i]
#                         category_counts[category] = category_counts.get(category, 0) + 1
#                         if category not in category_coals:
#                             category_coals[category] = []
#                         category_coals[category].append(i)

#             for category, constraints in self.category_constraints.items():
#                 ratio = category_ratios.get(category, 0)
#                 count = category_counts.get(category, 0)

#                 if ratio > 0:
#                     # Fix count violations
#                     if count > constraints['highest_count']:
#                         coal_indices = category_coals[category]
#                         # Remove coal with smallest ratio
#                         min_idx = min(coal_indices, key=lambda i: blend_array[i])
#                         blend_array[min_idx] = 0
#                         adjusted = True

#                     # Fix ratio violations
#                     if ratio < constraints['min_ratio']:
#                         # Increase this category
#                         if category in category_coals:
#                             idx = np.random.choice(category_coals[category])
#                             blend_array[idx] = blend_array[idx] + 1
#                             adjusted = True

#                     if ratio > constraints['max_ratio']:
#                         # Decrease this category
#                         if category in category_coals:
#                             idx = np.random.choice(category_coals[category])
#                             blend_array[idx] = max(0, blend_array[idx] - 1)
#                             adjusted = True

#             if not adjusted:
#                 # If no adjustment was made, we're stuck
#                 break

#         # Final normalization
#         blend_array = self.normalize_to_integer(blend_array)
#         return blend_array

#     def initialize_population(self) -> List[np.ndarray]:
#         """Initialize population with diverse feasible blends."""
#         population = []

#         # Add initial blends from CSV (converted to integers) - but check diversity
#         for idx, row in self.initial_blends.iterrows():
#             blend_array = row.values.astype(float)
#             if self.use_integer_ratios:
#                 blend_array = self.normalize_to_integer(blend_array)

#             if self.check_constraints(blend_array):
#                 sig = self.blend_to_signature(blend_array)
#                 if sig not in self.seen_blends:
#                     # Only add if diverse enough
#                     if self.is_diverse_enough(blend_array, population):
#                         population.append(blend_array)
#                         self.seen_blends.add(sig)

#         print(f"Added {len(population)} diverse blends from initial CSV")

#         # Generate diverse random feasible blends
#         max_attempts = self.population_size * 30
#         attempts = 0

#         while len(population) < self.population_size and attempts < max_attempts:
#             blend_array = self.generate_random_blend()

#             if self.check_constraints(blend_array):
#                 sig = self.blend_to_signature(blend_array)
#                 if sig not in self.seen_blends and self.is_diverse_enough(
#                     blend_array, population
#                 ):
#                     population.append(blend_array)
#                     self.seen_blends.add(sig)

#             attempts += 1

#         print(f"Generated {len(population) - len(population)} random diverse blends")

#         # If we don't have enough, generate with more aggressive mutation (relax diversity slightly)
#         attempts = 0
#         max_fill_attempts = self.population_size * 20

#         while len(population) < self.population_size and attempts < max_fill_attempts:
#             if len(population) > 0:
#                 base_blend = population[random.randint(0, len(population) - 1)].copy()
#                 mutated = self.aggressive_mutate(base_blend)

#                 if self.check_constraints(mutated):
#                     sig = self.blend_to_signature(mutated)
#                     if sig not in self.seen_blends:
#                         population.append(mutated)
#                         self.seen_blends.add(sig)
#             else:
#                 # Generate random if no valid blends yet
#                 blend = self.generate_random_blend()
#                 if self.check_constraints(blend):
#                     population.append(blend)

#             attempts += 1

#         return population

#     def evaluate_fitness(self, blend_array: np.ndarray) -> float:
#         """Evaluate fitness of a blend using cost function."""
#         try:
#             blend_dict = self.blend_array_to_dict(blend_array)

#             if not blend_dict:
#                 return float("inf")

#             # Get predictions
#             model_result = self.predictor.predict(blend_dict, verbose=False)

#             if model_result["status"] != "success":
#                 return float("inf")

#             # Get blend cost
#             blend_cost_result = self.predictor.Blend_cost(blend_dict)

#             # Calculate cost function
#             cost_input = {
#                 "Total blend Cost": blend_cost_result["Total blend Cost"],
#                 "predictions": model_result["predictions"],
#             }

#             cost_result = self.predictor.cost_function(cost_input)

#             return cost_result["Cost function evaluation"]

#         except Exception as e:
#             print(f"Error evaluating fitness: {e}")
#             return float("inf")

#     def tournament_selection(self, tournament_size: int = 5) -> np.ndarray:
#         """Select parent using tournament selection."""
#         tournament_indices = random.sample(range(len(self.population)), tournament_size)
#         tournament_fitness = [self.fitness_scores[i] for i in tournament_indices]
#         winner_idx = tournament_indices[np.argmin(tournament_fitness)]
#         return self.population[winner_idx].copy()

#     def crossover(
#         self, parent1: np.ndarray, parent2: np.ndarray
#     ) -> Tuple[np.ndarray, np.ndarray]:
#         """
#         Perform integer-based crossover between two parents.
#         """
#         if random.random() > self.crossover_rate:
#             return parent1.copy(), parent2.copy()

#         # Convert to integers
#         p1_int = np.round(parent1).astype(int)
#         p2_int = np.round(parent2).astype(int)

#         # Choose crossover method
#         method = random.choice(
#             ["uniform", "single_point", "two_point", "category_based"]
#         )

#         if method == "uniform":
#             # Uniform crossover
#             mask = np.random.rand(len(p1_int)) < 0.5
#             offspring1 = np.where(mask, p1_int, p2_int).astype(float)
#             offspring2 = np.where(mask, p2_int, p1_int).astype(float)

#         elif method == "single_point":
#             # Single point crossover
#             min_point = 1
#             max_point = len(p1_int) - 1
#             point = (
#                 random.randint(min_point, max_point)
#                 if max_point > min_point
#                 else min_point
#             )
#             offspring1 = np.concatenate([p1_int[:point], p2_int[point:]]).astype(float)
#             offspring2 = np.concatenate([p2_int[:point], p1_int[point:]]).astype(float)

#         elif method == "two_point":
#             # Two point crossover
#             if len(p1_int) > 2:
#                 points = sorted(random.sample(range(1, len(p1_int)), 2))
#                 offspring1 = np.concatenate(
#                     [
#                         p1_int[: points[0]],
#                         p2_int[points[0] : points[1]],
#                         p1_int[points[1] :],
#                     ]
#                 ).astype(float)
#                 offspring2 = np.concatenate(
#                     [
#                         p2_int[: points[0]],
#                         p1_int[points[0] : points[1]],
#                         p2_int[points[1] :],
#                     ]
#                 ).astype(float)
#             else:
#                 # Fallback to uniform if not enough genes
#                 mask = np.random.rand(len(p1_int)) < 0.5
#                 offspring1 = np.where(mask, p1_int, p2_int).astype(float)
#                 offspring2 = np.where(mask, p2_int, p1_int).astype(float)

#         else:  # category_based
#             # Mix entire categories from parents
#             offspring1 = p1_int.copy().astype(float)
#             offspring2 = p2_int.copy().astype(float)

#             for category in self.category_constraints.keys():
#                 if random.random() < 0.5:
#                     # Swap this category between offspring
#                     for i, coal in enumerate(self.coal_names):
#                         if self.category_map.get(coal) == category:
#                             offspring1[i], offspring2[i] = p2_int[i], p1_int[i]

#         # Normalize to 100
#         offspring1 = self.normalize_to_integer(offspring1)
#         offspring2 = self.normalize_to_integer(offspring2)

#         return offspring1, offspring2

#     def mutate(self, blend_array: np.ndarray) -> np.ndarray:
#         """Mutate a blend array with integer operations."""
#         if random.random() > self.mutation_rate:
#             return blend_array

#         mutated = np.round(blend_array).astype(int).astype(float)

#         # Choose mutation strategy
#         strategy = random.choice(["swap", "adjust", "redistribute", "add_remove"])

#         if strategy == "swap":
#             # Swap ratios between two coals
#             non_zero_indices = np.where(mutated > 0)[0]
#             if len(non_zero_indices) >= 2:
#                 i, j = random.sample(list(non_zero_indices), 2)
#                 mutated[i], mutated[j] = mutated[j], mutated[i]

#         elif strategy == "adjust":
#             # Adjust ratio of coals (transfer between two coals)
#             non_zero_indices = np.where(mutated > 0)[0]
#             if len(non_zero_indices) >= 2:
#                 donor_idx = random.choice(non_zero_indices)
#                 receiver_idx = random.choice(
#                     [i for i in non_zero_indices if i != donor_idx]
#                 )

#                 # Transfer 1-5 percentage points
#                 max_transfer = min(5, int(mutated[donor_idx] - 1))
#                 if max_transfer > 0:
#                     transfer = (
#                         random.randint(1, max_transfer) if max_transfer > 1 else 1
#                     )
#                     mutated[donor_idx] -= transfer
#                     mutated[receiver_idx] += transfer

#         elif strategy == "redistribute":
#             # Redistribute within a category
#             category = random.choice(list(self.category_constraints.keys()))
#             category_indices = [
#                 i
#                 for i, coal in enumerate(self.coal_names)
#                 if self.category_map.get(coal) == category and mutated[i] > 0
#             ]

#             if len(category_indices) >= 2:
#                 total = sum(mutated[i] for i in category_indices)
#                 num_coals = len(category_indices)

#                 # Random redistribution
#                 new_ratios = np.random.multinomial(
#                     int(total), [1 / num_coals] * num_coals
#                 )
#                 for idx, new_ratio in zip(category_indices, new_ratios):
#                     mutated[idx] = new_ratio

#         else:  # add_remove
#             # Try to add a new coal or remove an existing one
#             if random.random() < 0.5:
#                 # Add a new coal
#                 zero_indices = np.where(mutated == 0)[0]
#                 if len(zero_indices) > 0:
#                     new_coal_idx = random.choice(zero_indices)
#                     # Take from existing coals
#                     non_zero_indices = np.where(mutated > 3)[
#                         0
#                     ]  # Only from coals with >3%
#                     if len(non_zero_indices) > 0:
#                         min_amount = 2
#                         max_amount = 5
#                         amount = (
#                             random.randint(min_amount, max_amount)
#                             if max_amount > min_amount
#                             else min_amount
#                         )
#                         for _ in range(amount):
#                             donor_idx = random.choice(non_zero_indices)
#                             if mutated[donor_idx] > 1:
#                                 mutated[donor_idx] -= 1
#                                 mutated[new_coal_idx] += 1
#             else:
#                 # Remove a coal (set to 0 and redistribute)
#                 non_zero_indices = np.where(mutated > 0)[0]
#                 if len(non_zero_indices) > 3:  # Keep at least 3 coals
#                     remove_idx = random.choice(non_zero_indices)
#                     amount = int(mutated[remove_idx])
#                     mutated[remove_idx] = 0

#                     # Redistribute to remaining coals
#                     remaining = [i for i in non_zero_indices if i != remove_idx]
#                     if remaining:
#                         for _ in range(amount):
#                             receiver_idx = random.choice(remaining)
#                             mutated[receiver_idx] += 1

#         # Ensure still sums to 100
#         mutated = self.normalize_to_integer(mutated)

#         return mutated
#     def evolve(self):
#         """Run the genetic algorithm."""
#         print("Initializing population...")
#         self.population = self.initialize_population()

#         print(f"Evaluating initial population of {len(self.population)} blends...")
#         self.fitness_scores = [
#             self.evaluate_fitness(blend) for blend in self.population
#         ]

#         # Store population signatures with fitness
#         for blend, fitness in zip(self.population, self.fitness_scores):
#             sig = self.blend_to_signature(blend)
#             self.population_signatures[sig] = fitness

#         best_idx = np.argmin(self.fitness_scores)
#         self.best_fitness = self.fitness_scores[best_idx]
#         self.best_solution = self.population[best_idx].copy()

#         print(f"Initial best fitness: {self.best_fitness:.2f}")
#         print(f"Initial unique blends: {len(self.seen_blends)}")

#         for generation in range(self.generations):
#             # Sort population by fitness
#             sorted_indices = np.argsort(self.fitness_scores)
#             self.population = [self.population[i] for i in sorted_indices]
#             self.fitness_scores = [self.fitness_scores[i] for i in sorted_indices]

#             # Adaptive elite size - keep only truly unique elite
#             elite_population = []
#             elite_fitness = []

#             for i in range(min(self.elite_size * 2, len(self.population))):
#                 blend = self.population[i]
#                 fitness = self.fitness_scores[i]

#                 # Check if this elite is diverse enough from already selected elites
#                 if (
#                     self.is_diverse_enough(blend, elite_population)
#                     or len(elite_population) == 0
#                 ):
#                     elite_population.append(blend.copy())
#                     elite_fitness.append(fitness)

#                     if len(elite_population) >= self.elite_size:
#                         break

#             # Start new population with diverse elite
#             new_population = elite_population
#             new_fitness = elite_fitness

#             # Generate offspring
#             offspring_generated = 0
#             attempts = 0
#             max_attempts = self.population_size * 5

#             while (
#                 len(new_population) < self.population_size and attempts < max_attempts
#             ):
#                 attempts += 1

#                 # Select parents
#                 parent1 = self.tournament_selection()
#                 parent2 = self.tournament_selection()

#                 # Crossover
#                 offspring1, offspring2 = self.crossover(parent1, parent2)

#                 # Mutate
#                 offspring1 = self.mutate(offspring1)
#                 offspring2 = self.mutate(offspring2)

#                 # Repair if needed
#                 if not self.check_constraints(offspring1):
#                     offspring1 = self.repair_blend(offspring1)
#                 if not self.check_constraints(offspring2):
#                     offspring2 = self.repair_blend(offspring2)

#                 # Add to population if valid, unique, and diverse
#                 for offspring in [offspring1, offspring2]:
#                     if len(new_population) >= self.population_size:
#                         break

#                     if self.check_constraints(offspring):
#                         sig = self.blend_to_signature(offspring)

#                         # Check if this is a new unique blend
#                         if sig not in self.population_signatures:
#                             # Evaluate fitness
#                             fitness = self.evaluate_fitness(offspring)

#                             # Check diversity
#                             if (
#                                 self.is_diverse_enough(offspring, new_population)
#                                 or len(new_population) < self.elite_size * 2
#                             ):
#                                 new_population.append(offspring)
#                                 new_fitness.append(fitness)
#                                 self.seen_blends.add(sig)
#                                 self.population_signatures[sig] = fitness
#                                 offspring_generated += 1

#             # Fill remaining slots with aggressive mutations if needed
#             fill_attempts = 0
#             while (
#                 len(new_population) < self.population_size
#                 and fill_attempts < self.population_size * 3
#             ):
#                 fill_attempts += 1
#                 max_idx = min(20, len(self.population) - 1)
#                 base_idx = random.randint(0, max_idx) if max_idx > 0 else 0
#                 base = self.population[base_idx].copy()
#                 mutated = self.aggressive_mutate(base)

#                 if self.check_constraints(mutated):
#                     sig = self.blend_to_signature(mutated)
#                     if sig not in self.population_signatures:
#                         fitness = self.evaluate_fitness(mutated)
#                         new_population.append(mutated)
#                         new_fitness.append(fitness)
#                         self.seen_blends.add(sig)
#                         self.population_signatures[sig] = fitness

#             # Update population
#             self.population = new_population[: self.population_size]
#             self.fitness_scores = new_fitness[: self.population_size]

#             # Re-evaluate if we don't have fitness for all
#             while len(self.fitness_scores) < len(self.population):
#                 blend = self.population[len(self.fitness_scores)]
#                 fitness = self.evaluate_fitness(blend)
#                 self.fitness_scores.append(fitness)

#             # Update best solution
#             best_idx = np.argmin(self.fitness_scores)
#             if self.fitness_scores[best_idx] < self.best_fitness:
#                 self.best_fitness = self.fitness_scores[best_idx]
#                 self.best_solution = self.population[best_idx].copy()
#                 print(f"  → New best found! Fitness: {self.best_fitness:.2f}")

#             # Record history
#             avg_fitness = np.mean(self.fitness_scores)
#             self.history.append(
#                 {
#                     "generation": generation + 1,
#                     "best_fitness": self.best_fitness,
#                     "avg_fitness": avg_fitness,
#                     "worst_fitness": np.max(self.fitness_scores),
#                     "unique_blends": len(self.seen_blends),
#                     "population_diversity": len(self.population_signatures),
#                 }
#             )

#             if (generation + 1) % 10 == 0 or generation == 0:
#                 print(
#                     f"Gen {generation + 1}/{self.generations} - "
#                     f"Best: {self.best_fitness:.2f}, Avg: {avg_fitness:.2f}, "
#                     f"Unique: {len(self.population_signatures)}, New: {offspring_generated}"
#                 )

#         print(f"\nOptimization complete!")
#         print(f"Final best fitness: {self.best_fitness:.2f}")
#         print(f"Total unique blends explored: {len(self.seen_blends)}")
#         print(f"Unique blends in final analysis: {len(self.population_signatures)}")

#     def aggressive_mutate(self, blend_array: np.ndarray) -> np.ndarray:
#         """Apply multiple mutations for more diversity."""
#         mutated = blend_array.copy()
#         min_mutations = 2
#         max_mutations = 4
#         num_mutations = (
#             random.randint(min_mutations, max_mutations)
#             if max_mutations > min_mutations
#             else min_mutations
#         )

#         for _ in range(num_mutations):
#             mutated = self.mutate(mutated)

#         return mutated


#     def get_best_blend(self) -> Dict:
#         """Get the best blend found."""
#         if self.best_solution is None:
#             return None

#         blend_dict = self.blend_array_to_dict(self.best_solution)
#         model_result = self.predictor.predict(blend_dict, verbose=False)
#         blend_cost = self.predictor.Blend_cost(blend_dict)

#         return {
#             "blend": blend_dict,
#             "fitness": self.best_fitness,
#             "predictions": model_result["predictions"],
#             "category_distribution": model_result["category_distribution"],
#             "blend_cost": blend_cost["Total blend Cost"],
#         }

#     def get_top_n_blends(self, n: int = 5, min_diversity: float = None) -> List[Dict]:
#         """
#         Get top N best unique and diverse blends.

#         Args:
#             n: Number of top blends to return
#             min_diversity: Minimum diversity required between returned blends (defaults to self.diversity_threshold)
#         """
#         if min_diversity is None:
#             min_diversity = self.diversity_threshold

#         # Get all unique blends with their fitness
#         unique_blends = []
#         seen_sigs = set()

#         sorted_indices = np.argsort(self.fitness_scores)

#         for idx in sorted_indices:
#             sig = self.blend_to_signature(self.population[idx])
#             if sig not in seen_sigs:
#                 unique_blends.append(idx)
#                 seen_sigs.add(sig)

#         # Now select diverse blends from the unique ones
#         diverse_blends = []
#         diverse_blend_arrays = []

#         for idx in unique_blends:
#             blend = self.population[idx]

#             # Check if this blend is diverse enough from already selected blends
#             is_diverse = True
#             for selected_blend in diverse_blend_arrays:
#                 diversity = self.calculate_diversity(blend, selected_blend)
#                 if diversity < min_diversity:
#                     is_diverse = False
#                     break

#             if is_diverse or len(diverse_blends) == 0:
#                 diverse_blends.append(idx)
#                 diverse_blend_arrays.append(blend)

#             if len(diverse_blends) >= n:
#                 break

#         # Build result list
#         top_blends = []
#         for idx in diverse_blends:
#             blend_dict = self.blend_array_to_dict(self.population[idx])
#             model_result = self.predictor.predict(blend_dict, verbose=False)
#             blend_cost = self.predictor.Blend_cost(blend_dict)

#             top_blends.append(
#                 {
#                     "blend": blend_dict,
#                     "fitness": self.fitness_scores[idx],
#                     "predictions": model_result["predictions"],
#                     "category_distribution": model_result["category_distribution"],
#                     "blend_cost": blend_cost["Total blend Cost"],
#                 }
#             )

#         return top_blends


# # Example usage:
# if __name__ == "__main__":
#     predictor = CoalBlendPredictor()

#     ga = CoalBlendGA(
#         best_blend_csv="Best_blend_ratio.csv",
#         clean_exist_coals_csv="Clean_exist_coals.csv",
#         predictor=predictor,
#         population_size=150,
#         generations=75,
#     )
#     print(CoalBlendGA)
#     print(dir(CoalBlendGA))

#     # Run optimization
#     ga.evolve()

#     # Get best blend
#     best_blend = ga.get_best_blend()
#     print("\nBest Blend Found:")
#     print(f"Blend: {best_blend['blend']}")
#     print(f"Predictions: {best_blend['predictions']}")
#     print(f"Cost: {best_blend['blend_cost']}")

#     # Get top 5 blends
#     top_blends = ga.get_top_n_blends(5)
#     for i, blend in enumerate(top_blends, 1):
#         print(f"\nTop {i} Blend:")
#         print(f"Fitness: {blend['fitness']:.2f}")
#         print(f"Predictions: {blend['predictions']}")


Streaming output truncated to the last 5000 lines.
{'blend_composition': {'Moranbah North': 19.0, 'Goonyella': 19.0, 'Metropolitan': 24.0, 'Eagle crrek': 13.0, 'Daunia (SHCC)': 8.0, 'Blue creek': 17.0}, 'category_distribution': {'HCC': 38.0, 'SHCC': 45.0, 'HFCC': 17.0, 'PCI': 0, 'WC': 0}, 'predictions': {'CRI': 24.88088877456481, 'CSR': 65.26908512628276, 'ASH': 14.418115153528298, 'VM': 0.7990924687284771}, 'status': 'success'}
{'blend_composition': {'Moranbah North': 14.0, 'Dhamra SHCC PDN': 10.0, 'Daunia (SHCC)': 33.0, 'Mt. Laurel': 19.0, 'North Goonyella': 24.0}, 'category_distribution': {'HCC': 38.0, 'SHCC': 43.0, 'HFCC': 19.0, 'PCI': 0, 'WC': 0}, 'predictions': {'CRI': 24.865029450370777, 'CSR': 65.31402729908854, 'ASH': 14.403935261477232, 'VM': 0.7957267180253575}, 'status': 'success'}
{'blend_composition': {'Caval Ridge': 19.0, 'PDN': 28.0, 'Eagle crrek': 23.0, 'Dhamra SHCC PDN': 11.0, 'Uvalnaya': 19.0}, 'category_distribution': {'HCC': 47.0, 'SHCC': 34.0, 'HFCC': 19.0, 'PCI':